In [1]:
# Libraries
import mysql.connector
import pandas as pd
import json
import numpy as np
import calendar

TRAN_CODE = '738'
YEAR = "2022"
MONTH = "01"

In [2]:
# MYSQL Connection string

config = {
  'user': 'root',
  'password': 'Nls@123',
  'host': '127.0.0.1',
  'database': 'nlscore_sidian',
  'auth_plugin' : 'mysql_native_password',
  'raise_on_warnings': True
}

In [3]:
num_days = calendar.monthrange(int(YEAR), int(MONTH))[1]
START_DATE = YEAR + '-' + MONTH + '-' + '01'
END_DATE = YEAR + '-' + MONTH + '-' + str(num_days)

In [4]:
# selecting the transaction code that should be charged
mysql_connection = mysql.connector.connect(**config)
TRANSACTION_CHARGE_CODES = pd.read_sql('select * from transaction$charge$codes where TRANSACTION_CODE = '+TRAN_CODE+'', con=mysql_connection)
mysql_connection.close()
TRANSACTION_CHARGE_CODES.head()

,ID,TRANSACTION_TYPE,TRANSACTION_NAME,TRANSACTION_CODE,CHARGE_TRANSACTION_CODE,TAX_TRANSACTION_CODE,CHARGE_CODE,TAX
0,6,IB,IB_PESALINK-70,738,234,465,KITSCOMM,20
1,7,IB,IB_PESALINK-70,738,51,None,PESALINKCHG,None


In [5]:
# list which includes the transaction code,charge code,tax code
TRANSACTION_CODE = TRANSACTION_CHARGE_CODES['TRANSACTION_CODE'].unique().tolist()
CHARGE_CODE = [TRANSACTION_CHARGE_CODES['CHARGE_TRANSACTION_CODE'][0]]
TAX_CODE = TRANSACTION_CHARGE_CODES['TAX_TRANSACTION_CODE'].unique().tolist()

TAX_CODE = [x for x in TAX_CODE if x is not None]
print('TRANSACTION_CODE - ',TRANSACTION_CODE)
print('CHARGE_CODE - ',CHARGE_CODE)
print('TAX_CODE - ',TAX_CODE)

TRANSACTION_CODE -  ['738']
CHARGE_CODE -  ['234']
TAX_CODE -  ['465']


In [6]:
# converting codes to list
CODES = TRANSACTION_CODE + CHARGE_CODE + TAX_CODE
print('CODES - ',CODES)

CODES -  ['738', '234', '465']


In [7]:
# converting list to string
FILTER_CODES = ','.join(CODES)

FILTER_CODES

'738,234,465'

In [8]:
# selecting the data using filter codes
mysql_connection = mysql.connector.connect(**config)
df = pd.read_sql('SELECT AMOUNT_LCY,TRANSACTION_CODE,TRANS_REFERENCE,TIME_STAMP from statement$entry_2022_jan12 where cast(TIME_STAMP as date) between "'+str(START_DATE)+'" and "'+str(END_DATE)+'" and TRANSACTION_CODE in ('+FILTER_CODES+')', con=mysql_connection)
mysql_connection.close()
df.tail()

,AMOUNT_LCY,TRANSACTION_CODE,TRANS_REFERENCE,TIME_STAMP
237428,2.0,465,CHG2013001702BI,2022-01-13 02:53:48
237429,2.0,465,CHG2013000359CJ,2022-01-13 02:35:25
237430,2.0,465,CHG2013002731TQ,2022-01-13 03:02:34
237431,-2.0,234,FTN2013003034ZH,2022-01-13 03:02:34
237432,2.0,465,FTN2013003034ZH,2022-01-13 03:02:34


In [9]:
# dataframe info
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 237433 entries, 0 to 237432
Data columns (total 4 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   AMOUNT_LCY        237433 non-null  float64       
 1   TRANSACTION_CODE  237433 non-null  object        
 2   TRANS_REFERENCE   237433 non-null  object        
 3   TIME_STAMP        237433 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 33.5 MB


In [10]:
# removing annotations from trans reference
df['TRANS_REFERENCE'] = df['TRANS_REFERENCE'].str.split("\\").str[0]

In [11]:
# filtering transaction charge code data's for trans reference
TRANS_REFERENCE = df[df['TRANSACTION_CODE'].isin(TRANSACTION_CODE)]
TRANS_REFERENCE = TRANS_REFERENCE[['TRANS_REFERENCE']]
TRANS_REFERENCE.reset_index(drop=True, inplace=True)

In [12]:
# converting trans reference as list
TRANS_REFERENCE = TRANS_REFERENCE['TRANS_REFERENCE'].unique().tolist()

In [13]:
# filtering charge code data's using trans reference
df = df[df['TRANS_REFERENCE'].isin(TRANS_REFERENCE)]
df.reset_index(drop=True, inplace=True)
df.head()

,AMOUNT_LCY,TRANSACTION_CODE,TRANS_REFERENCE,TIME_STAMP
0,-83.6,234,FTN1365221627DR,2022-01-01 01:35:57
1,-27500.0,738,FTN1365221627DR,2022-01-01 01:35:57
2,12.0,465,FTN1365221627DR,2022-01-01 01:35:57
3,-47.6,234,FTN1365214111SS,2022-01-01 01:09:19
4,-5000.0,738,FTN1365214111SS,2022-01-01 01:09:19


In [14]:
# total no of unique chragable transations
print("Total TRANS_REFERENCE count - ",len(df['TRANS_REFERENCE'].unique().tolist()))

Total TRANS_REFERENCE count -  2363


In [15]:
# creating list for map the code with respective transactions
TRANSACTION_CODES = [TRANSACTION_CODE,CHARGE_CODE,TAX_CODE]
print('TRANSACTION_CODES -',TRANSACTION_CODES)
TRANSACTION_TYPES = ['TRANSACTION_AMOUNT','CHARGE_AMOUNT','TAX_AMOUNT']
print('TRANSACTION_TYPES -',TRANSACTION_TYPES)

TRANSACTION_CODES - [['738'], ['234'], ['465']]
TRANSACTION_TYPES - ['TRANSACTION_AMOUNT', 'CHARGE_AMOUNT', 'TAX_AMOUNT']


In [16]:
# json data creation for mapping code with respective transactions
data = {}
for i in range(0,len(TRANSACTION_TYPES)):
    for j in range(0,len(TRANSACTION_CODES[i])):
        data[TRANSACTION_CODES[i][j]] = TRANSACTION_TYPES[i]
data

{'738': 'TRANSACTION_AMOUNT', '234': 'CHARGE_AMOUNT', '465': 'TAX_AMOUNT'}

In [17]:
# Creating Transaction type column
df['TRANSACTION_TYPE'] = df['TRANSACTION_CODE'].map(data)
df.head()

,AMOUNT_LCY,TRANSACTION_CODE,TRANS_REFERENCE,TIME_STAMP,TRANSACTION_TYPE
0,-83.6,234,FTN1365221627DR,2022-01-01 01:35:57,CHARGE_AMOUNT
1,-27500.0,738,FTN1365221627DR,2022-01-01 01:35:57,TRANSACTION_AMOUNT
2,12.0,465,FTN1365221627DR,2022-01-01 01:35:57,TAX_AMOUNT
3,-47.6,234,FTN1365214111SS,2022-01-01 01:09:19,CHARGE_AMOUNT
4,-5000.0,738,FTN1365214111SS,2022-01-01 01:09:19,TRANSACTION_AMOUNT


In [18]:
# pivoting the table using TRANS_REFERENCE
df_pivot = df.pivot_table(index=['TRANS_REFERENCE'], columns='TRANSACTION_TYPE', values='AMOUNT_LCY').reset_index()
df_pivot.fillna(0, inplace=True)
df_pivot.tail()

TRANSACTION_TYPE,TRANS_REFERENCE,CHARGE_AMOUNT,TAX_AMOUNT,TRANSACTION_AMOUNT
2358,FTN220112IXTKBL,-59.6,8.0,-20000.0
2359,FTN220112IXTKBM,-83.6,12.0,-22500.0
2360,FTN220112IXTKBN,-59.6,8.0,-12500.0
2361,FTN220112IXTKBO,-59.6,8.0,-15000.0
2362,FTN220112IXTKEK,-203.6,32.0,-410000.0


In [19]:
# merging 2 dataframes
df = pd.merge(df, df_pivot, on=['TRANS_REFERENCE'])
# filtering duplicates by keeping trans reference
df = df[df['TRANSACTION_TYPE'].isin(['TRANSACTION_AMOUNT'])]
df.reset_index(drop=True, inplace=True)
# dropping duplicate values
df = df.drop_duplicates(["TRANS_REFERENCE"],ignore_index = True)
# # converting transaction amount negative sign to positive sign
df['TRANSACTION_AMOUNT'] = df['TRANSACTION_AMOUNT'].abs()
df.head()

,AMOUNT_LCY,TRANSACTION_CODE,TRANS_REFERENCE,TIME_STAMP,TRANSACTION_TYPE,CHARGE_AMOUNT,TAX_AMOUNT,TRANSACTION_AMOUNT
0,-27500.0,738,FTN1365221627DR,2022-01-01 01:35:57,TRANSACTION_AMOUNT,-83.6,12.0,27500.0
1,-5000.0,738,FTN1365214111SS,2022-01-01 01:09:19,TRANSACTION_AMOUNT,-47.6,6.0,5000.0
2,-245000.0,738,FTN1365232946SM,2022-01-01 02:32:04,TRANSACTION_AMOUNT,-203.6,32.0,245000.0
3,-144500.0,738,FTN1365211429IS,2022-01-01 00:36:48,TRANSACTION_AMOUNT,-155.6,24.0,144500.0
4,-17000.0,738,FTN1365211918AU,2022-01-01 00:39:30,TRANSACTION_AMOUNT,-59.6,8.0,17000.0


In [20]:
df = df[['TRANS_REFERENCE','TRANSACTION_CODE','TRANSACTION_AMOUNT','CHARGE_AMOUNT','TAX_AMOUNT','TIME_STAMP']]
df.tail()

,TRANS_REFERENCE,TRANSACTION_CODE,TRANSACTION_AMOUNT,CHARGE_AMOUNT,TAX_AMOUNT,TIME_STAMP
2358,FTN2012162851GW,738,360000.0,-203.6,32.0,2022-01-12 19:01:03
2359,FTN2012173203JJ,738,84755.0,-107.6,16.0,2022-01-12 20:02:42
2360,FTN2012190329QJ,738,100000.0,-107.6,16.0,2022-01-12 21:34:16
2361,FTN2012205826WY,738,240000.0,-203.6,32.0,2022-01-12 23:29:59
2362,FTN2012212958JM,738,170000.0,-155.6,24.0,2022-01-13 00:02:42


In [21]:
# re validating total no of unique chragable transations
print("Total TRANS_REFERENCE count - ",len(df['TRANS_REFERENCE'].unique().tolist()))

Total TRANS_REFERENCE count -  2363


In [22]:
# for i in range(0,len(df)):
#     if(df['CHARGE_AMOUNT'][i] != 0):
#         df['CHARGE_AMOUNT'][i] = df['CHARGE_AMOUNT'][i] + df['TAX_AMOUNT'][i]
#     else:
#         df['CHARGE_AMOUNT'][i] = df['CHARGE_AMOUNT'][i]

In [23]:
df.loc[(df['CHARGE_AMOUNT'] != 0),'CHARGE_AMOUNT'] = df[['CHARGE_AMOUNT','TAX_AMOUNT']].sum(axis=1)
df.loc[(df['CHARGE_AMOUNT'] == 0), 'CHARGE_AMOUNT'] = df['CHARGE_AMOUNT']
# # converting charge amount negative sign to positive sign
df['CHARGE_AMOUNT'] = df['CHARGE_AMOUNT'].abs()
df.head()

,TRANS_REFERENCE,TRANSACTION_CODE,TRANSACTION_AMOUNT,CHARGE_AMOUNT,TAX_AMOUNT,TIME_STAMP
0,FTN1365221627DR,738,27500.0,71.6,12.0,2022-01-01 01:35:57
1,FTN1365214111SS,738,5000.0,41.6,6.0,2022-01-01 01:09:19
2,FTN1365232946SM,738,245000.0,171.6,32.0,2022-01-01 02:32:04
3,FTN1365211429IS,738,144500.0,131.6,24.0,2022-01-01 00:36:48
4,FTN1365211918AU,738,17000.0,51.6,8.0,2022-01-01 00:39:30


In [24]:
# selecting the transaction code charges
mysql_connection = mysql.connector.connect(**config)
#df_charges = pd.read_sql('select tcc.TRANSACTION_CODE,bct.CALCULATION_BASIS,bct.FLAT_AMOUNT,bct.CALC_TYPE,bct.PERCENTAGE,bct.UPTO_AMOUNT,bct.MAXIMUM_AMOUNT,bct.MAXIMUM_AMOUNT,tcc.TAX from bnk_charge$types bct inner join transaction$charge$codes tcc on bct.id = tcc.CHARGE_CODE where tcc.TRANSACTION_CODE = 738', con=mysql_connection)
df_charges = pd.read_sql('select bct.id,tcc.TRANSACTION_CODE,bct.CALCULATION_BASIS,bct.FLAT_AMOUNT,bct.CALC_TYPE,bct.PERCENTAGE,bct.UPTO_AMOUNT,bct.MINIMUM_AMOUNT,bct.MAXIMUM_AMOUNT,tcc.TAX from bnk_charge$types bct inner join transaction$charge$codes tcc on bct.id = tcc.CHARGE_CODE where tcc.TRANSACTION_CODE =  '+TRAN_CODE+' ', con=mysql_connection)
mysql_connection.close()
df_charges.head()

,id,TRANSACTION_CODE,CALCULATION_BASIS,FLAT_AMOUNT,CALC_TYPE,PERCENTAGE,UPTO_AMOUNT,MINIMUM_AMOUNT,MAXIMUM_AMOUNT,TAX
0,KITSCOMM,738,PERCENTAGE,None,LEVEL^LEVEL^LEVEL^LEVEL^LEVEL^LEVEL^LEVEL^LEVE...,0.00001^0.00001^0.00001^0.00001^0.00001^0.0000...,501.00^5001.00^10001.00^20001.00^50001.00^1000...,0.00^8.40^18.40^28.40^48.40^68.40^108.40^148.4...,None,20
1,PESALINKCHG,738,PERCENTAGE,None,LEVEL^LEVEL,0.00001^0.00001,^501.00,0^11.60,None,None


In [27]:
# df_charges['UPTO_AMOUNT'][1][0] == '^'
if(df_charges['FLAT_AMOUNT'][0] == None):
    for column in df_charges:
        for i in range(0,len(df_charges)):
            if(df_charges[column][i] != None):
                if(str(df_charges[column][i][0]) == '^'):
                    df_charges[column][i] =  df_charges[column][i][1:]           
df_charges.head()

,id,TRANSACTION_CODE,CALCULATION_BASIS,FLAT_AMOUNT,CALC_TYPE,PERCENTAGE,UPTO_AMOUNT,MINIMUM_AMOUNT,MAXIMUM_AMOUNT,TAX
0,KITSCOMM,738,PERCENTAGE,None,LEVEL^LEVEL^LEVEL^LEVEL^LEVEL^LEVEL^LEVEL^LEVE...,0.00001^0.00001^0.00001^0.00001^0.00001^0.0000...,501.00^5001.00^10001.00^20001.00^50001.00^1000...,0.00^8.40^18.40^28.40^48.40^68.40^108.40^148.4...,None,20
1,PESALINKCHG,738,PERCENTAGE,None,LEVEL^LEVEL,0.00001^0.00001,501.00,0^11.60,None,None


In [28]:
def fn_str_split(string):
    return string.split('^')

In [29]:
cols_to_df_list = ['CALC_TYPE','PERCENTAGE','UPTO_AMOUNT','MINIMUM_AMOUNT','MAXIMUM_AMOUNT','id','TAX']
df_chg = pd.DataFrame(columns = cols_to_df_list)
if(df_charges['FLAT_AMOUNT'][0] == None):
    
    for j in range (0,len(df_charges)):
        df_len = len(df_chg)
        for i in cols_to_df_list:
            if df_charges[i][j]:
                split_list = fn_str_split(df_charges[i][j])
                for k in range (0,len(split_list)):
                    if((i == "id") | (i == 'TAX')):
                        df_chg.loc[df_len + k,i] = df_charges[i][j]
                    else:
                        df_chg.loc[df_len + k,i] = split_list[k]
    
    
    df_chg['id'] = df_chg[['id']].ffill()
    df_chg['TAX'] = df_chg.groupby('id')['TAX'].ffill()
    df_chg=df_chg.replace(r'', 0, regex=True)
    
    
    
    df_chg['PERCENTAGE'] = df_chg['PERCENTAGE'].astype('float')
    df_chg['MAX_AMT'] = df_chg['UPTO_AMOUNT'].astype('float')
    df_chg['MIN_CHARGE'] = df_chg['MINIMUM_AMOUNT'].astype('float')
    df_chg['MAX_CHARGE'] = df_chg['MAXIMUM_AMOUNT'].astype('float')
    df_chg['TAX'] = df_chg['TAX'].astype('float')
    
    
    df_chg['MIN_AMT'] = 0
    for i in range(0,len(df_chg)):
        if(i == 0):
            df_chg['MIN_AMT'][i] =  0
        else:
            df_chg['MIN_AMT'][i] =  df_chg['MAX_AMT'][i-1] + 1

    df_chg.loc[(df_chg['MIN_AMT'].isnull()) | (df_chg['MIN_AMT'] > df_chg['MAX_AMT']),'MIN_AMT'] = 0
    df_chg = df_chg[["id","CALC_TYPE","PERCENTAGE","MIN_AMT","MAX_AMT","MIN_CHARGE","MAX_CHARGE","TAX"]]
    
df_chg

/tmp/ipykernel_16824/1569325905.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_chg['MIN_AMT'][i] =  0
/tmp/ipykernel_16824/1569325905.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_chg['MIN_AMT'][i] =  df_chg['MAX_AMT'][i-1] + 1
/home/sabarish/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,id,CALC_TYPE,PERCENTAGE,MIN_AMT,MAX_AMT,MIN_CHARGE,MAX_CHARGE,TAX
0,KITSCOMM,LEVEL,0.00001,0.0,501.0,0.0,NaN,20.0
1,KITSCOMM,LEVEL,0.00001,502.0,5001.0,8.4,NaN,20.0
2,KITSCOMM,LEVEL,0.00001,5002.0,10001.0,18.4,NaN,20.0
3,KITSCOMM,LEVEL,0.00001,10002.0,20001.0,28.4,NaN,20.0
4,KITSCOMM,LEVEL,0.00001,20002.0,50001.0,48.4,NaN,20.0
5,KITSCOMM,LEVEL,0.00001,50002.0,100001.0,68.4,NaN,20.0
6,KITSCOMM,LEVEL,0.00001,100002.0,200001.0,108.4,NaN,20.0
7,KITSCOMM,LEVEL,0.00001,200002.0,500001.0,148.4,NaN,20.0
8,KITSCOMM,LEVEL,0.00001,500002.0,NaN,188.4,NaN,20.0
9,PESALINKCHG,LEVEL,0.00001,0.0,501.0,0.0,NaN,NaN


In [27]:
# For FLAT AMOUNT with no history
data = {}
for i in range(0,len(df_charges)):
    if(df_charges['FLAT_AMOUNT'][i] != None):
        # Creating expected charge column
        data[df_charges['TRANSACTION_CODE'][i]] = df_charges['FLAT_AMOUNT'][i]
        df['EXPECTED_CHARGE'] = df['TRANSACTION_CODE'].map(data)
        # Creating expected Tax percentage
        data[df_charges['TRANSACTION_CODE'][i]] = df_charges['TAX'][i]
        df['EXPECTED_TAX'] = df['TRANSACTION_CODE'].map(data)
        #conversion
        df['EXPECTED_CHARGE'] = df['EXPECTED_CHARGE'].astype('float')
        df['EXPECTED_TAX'] = df['EXPECTED_TAX'].astype('float')
        # expected tax calculation
        df['EXPECTED_TAX'] = df['EXPECTED_CHARGE'] /100 * df['EXPECTED_TAX']
        
        
    elif((df_charges['CALC_TYPE'][i] == 'LEVEL') | (df_charges['CALC_TYPE'][i] == 'BAND')):
        # Creating expected charge percentage
        data[df_charges['TRANSACTION_CODE'][i]] = df_charges['PERCENTAGE'][i]
        df['EXPECTED_CHARGE'] = df['TRANSACTION_CODE'].map(data)
        # Creating expected Tax percentage
        data[df_charges['TRANSACTION_CODE'][i]] = df_charges['TAX'][i]
        df['EXPECTED_TAX'] = df['TRANSACTION_CODE'].map(data)
        #conversion
        df['EXPECTED_CHARGE'] = df['EXPECTED_CHARGE'].astype('float')
        df['EXPECTED_TAX'] = df['EXPECTED_TAX'].astype('float')
        df_charges['MINIMUM_AMOUNT'] = df_charges['MINIMUM_AMOUNT'].astype('float')
        df_charges['MAXIMUM_AMOUNT'] = df_charges['MAXIMUM_AMOUNT'].astype('float')
        # expected charge and tax calculation
        df['EXPECTED_CHARGE'] = round(df['TRANSACTION_AMOUNT'] /100 * df['EXPECTED_CHARGE'],2)
        df['EXPECTED_TAX'] = round(df['EXPECTED_CHARGE'] /100 * df['EXPECTED_TAX'],2)
        # comparing charges with minimum maximum amount
        # for charge lesser then minimum charge amount
        if(pd.isnull(df_charges.iloc[i]['MINIMUM_AMOUNT']) == False):
            df.loc[(df['EXPECTED_CHARGE'] < df_charges['MINIMUM_AMOUNT'][i]) , 'EXPECTED_CHARGE'] =  df_charges['MINIMUM_AMOUNT'][i]
        # for charge greter then minimum charge amount
        if(pd.isnull(df_charges.iloc[i]['MAXIMUM_AMOUNT']) == False):
            df.loc[(df['EXPECTED_CHARGE'] > df_charges['MAXIMUM_AMOUNT'][i]) , 'EXPECTED_CHARGE'] =  df_charges['MAXIMUM_AMOUNT'][i]      

In [28]:
# for level charge with no hostory
if(df_charges['FLAT_AMOUNT'][i] == None):
    if(df_chg['CALC_TYPE'][0] == "LEVEL"):
        # for expected charge initialization
        df['EXPECTED_CHARGE'] = 0
        df['EXPECTED_TAX'] = 0

        for i in range (0,len(df_chg)):
            # for charge as per percentage
            df.loc[(df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df['TRANSACTION_AMOUNT'] <= df_chg['MAX_AMT'][i]), 'CHARGE'] = round((df['TRANSACTION_AMOUNT'] * df_chg['PERCENTAGE'][i] /100),2)
            # for charge lesser then minimum charge amount
            if(pd.isnull(df_chg.iloc[i]['MIN_CHARGE']) == False):
                df.loc[(df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df['TRANSACTION_AMOUNT'] <= df_chg['MAX_AMT'][i]) & (df['CHARGE'] < df_chg['MIN_CHARGE'][i]) , 'CHARGE'] =  df_chg['MIN_CHARGE'][i]
            # for charge greter then minimum charge amount
            if(pd.isnull(df_chg.iloc[i]['MAX_CHARGE']) == False):
                df.loc[(df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df['TRANSACTION_AMOUNT'] <= df_chg['MAX_AMT'][i]) & (df['CHARGE'] > df_chg['MAX_CHARGE'][i]) , 'CHARGE'] =  df_chg['MAX_CHARGE'][i]
            # for tax as per percentage
            if(pd.isnull(df_chg.iloc[i]['TAX']) == False):
                df.loc[(df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df['TRANSACTION_AMOUNT'] <= df_chg['MAX_AMT'][i]), 'TAX'] = round((df['CHARGE'] * df_chg['TAX'][i] /100),2)

            # for expected charge
            df.loc[(df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df['TRANSACTION_AMOUNT'] <= df_chg['MAX_AMT'][i]), 'EXPECTED_CHARGE'] = round((df['EXPECTED_CHARGE'] + df['CHARGE']),2)
            # for expected tax
            if(pd.isnull(df_chg.iloc[i]['TAX']) == False):
                df.loc[(df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df['TRANSACTION_AMOUNT'] <= df_chg['MAX_AMT'][i]), 'EXPECTED_TAX'] = round((df['EXPECTED_TAX'] + df['TAX']),2)

        df_chg = df_chg.loc[(df_chg['MAX_AMT'].isnull())]
        df_chg.reset_index(drop=True, inplace=True)

        for i in range (0,len(df_chg)):
            # for charge as per percentage
            df.loc[(df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'CHARGE'] = round((df['TRANSACTION_AMOUNT'] * df_chg['PERCENTAGE'][i] /100),2)
            # for charge lesser then minimum charge amount
            if(pd.isnull(df_chg.iloc[i]['MIN_CHARGE']) == False):
                df.loc[(df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df['CHARGE'] < df_chg['MIN_CHARGE'][i]) , 'CHARGE'] =  df_chg['MIN_CHARGE'][i]
            # for charge greter then minimum charge amount
            if(pd.isnull(df_chg.iloc[i]['MAX_CHARGE']) == False):
                df.loc[(df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df['CHARGE'] > df_chg['MAX_CHARGE'][i]) , 'CHARGE'] =  df_chg['MAX_CHARGE'][i]
            # for tax as per percentage
            if(pd.isnull(df_chg.iloc[i]['TAX']) == False):
                df.loc[(df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'TAX'] = round((df['CHARGE'] * df_chg['TAX'][i] /100),2)

            # for expected charge
            df.loc[(df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'EXPECTED_CHARGE'] = round((df['EXPECTED_CHARGE'] + df['CHARGE']),2)
            # for expected tax
            if(pd.isnull(df_chg.iloc[i]['TAX']) == False):
                df.loc[(df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'EXPECTED_TAX'] = round((df['EXPECTED_TAX'] + df['TAX']),2)  

In [29]:
# for band charge with no hostory
if(df_charges['FLAT_AMOUNT'][i] == None):
    if(df_chg['CALC_TYPE'][0] == "BAND"): 
        # for expected charge initialization
        df['EXPECTED_CHARGE'] = 0
        df['EXPECTED_TAX'] = 0

        for i in range (0,len(df_chg)):
            # Chargable amount calculation
            df.loc[(df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'CHARGABLE_AMT'] = df['TRANSACTION_AMOUNT'] - (df_chg['MIN_AMT'][i])
            if(pd.isnull(df_chg.iloc[i]['MAX_AMT']) == False):
                df.loc[(df['CHARGABLE_AMT'] > (df_chg['MAX_AMT'][i] - df_chg['MIN_AMT'][i])), 'CHARGABLE_AMT'] = df_chg['MAX_AMT'][i] - df_chg['MIN_AMT'][i]
            else:
                df.loc[(df['TRANSACTION_AMOUNT'] > (df_chg['MIN_AMT'][i])), 'CHARGABLE_AMT'] = df['TRANSACTION_AMOUNT'] - (df_chg['MIN_AMT'][i])
            # for charge as per percentage
            df.loc[(df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'CHARGE'] = round((df['CHARGABLE_AMT'] * df_chg['PERCENTAGE'][i] /100),2)
            # for tax as per percentage
            if(pd.isnull(df_chg.iloc[i]['TAX']) == False):
                df.loc[(df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'TAX'] = round((df['CHARGE'] * df_chg['TAX'][i] /100),2)

            # for expected charge
            df.loc[(df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'EXPECTED_CHARGE'] = round((df['EXPECTED_CHARGE'] + df['CHARGE']),2)
            # for expected tax
            if(pd.isnull(df_chg.iloc[i]['TAX']) == False):
                df.loc[(df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'EXPECTED_TAX'] = round((df['EXPECTED_TAX'] + df['TAX']),2)

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2363 entries, 0 to 2362
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   TRANS_REFERENCE     2363 non-null   object        
 1   TRANSACTION_CODE    2363 non-null   object        
 2   TRANSACTION_AMOUNT  2363 non-null   float64       
 3   CHARGE_AMOUNT       2363 non-null   float64       
 4   TAX_AMOUNT          2363 non-null   float64       
 5   TIME_STAMP          2363 non-null   datetime64[ns]
 6   EXPECTED_CHARGE     2363 non-null   float64       
 7   EXPECTED_TAX        2363 non-null   float64       
 8   CHARGE              2363 non-null   float64       
 9   TAX                 2363 non-null   float64       
dtypes: datetime64[ns](1), float64(7), object(2)
memory usage: 184.7+ KB


In [31]:
# # json data creation for transaction codes with respective Tax
# data = {}
# for i in range(0,len(df_charges)):
#     if(df_charges['FLAT_AMOUNT'][i] != None):
#         data[df_charges['TRANSACTION_CODE'][i]] = df_charges['TAX'][i]
# data

In [32]:
# # Creating expected Tax column
# df['EXPECTED_TAX'] = df['TRANSACTION_CODE'].map(data)
# df.head()

In [33]:
# data type conversion
df['CHARGE_AMOUNT'] = df['CHARGE_AMOUNT'].astype('float')
df['TAX_AMOUNT'] = df['TAX_AMOUNT'].astype('float')
# df['EXPECTED_CHARGE'] = df['EXPECTED_CHARGE'].astype('float')
# df['EXPECTED_TAX'] = df['EXPECTED_TAX'].astype('float')

In [34]:
# # actual tax calculation
# df['EXPECTED_TAX'] = df['EXPECTED_CHARGE'] /100 * df['EXPECTED_TAX']
# df.head()

In [35]:
# charge_deviation calculation
df['CHARGE_DEVIATION'] = df['CHARGE_AMOUNT'] - df['EXPECTED_CHARGE']
# tax_deviation calculation
df['TAX_DEVIATION'] = df['TAX_AMOUNT'] - df['EXPECTED_TAX']
# # overall deviation
# df['OVERALL_DEVIATION'] = df['CHARGE_DEVIATION'] + df['TAX_DEVIATION']
df.head()

,TRANS_REFERENCE,TRANSACTION_CODE,TRANSACTION_AMOUNT,CHARGE_AMOUNT,TAX_AMOUNT,TIME_STAMP,EXPECTED_CHARGE,EXPECTED_TAX,CHARGE,TAX,CHARGE_DEVIATION,TAX_DEVIATION
0,FTN1365221627DR,738,27500.0,71.6,12.0,2022-01-01 01:35:57,60.0,9.68,11.6,9.68,11.6,2.32
1,FTN1365214111SS,738,5000.0,41.6,6.0,2022-01-01 01:09:19,20.0,1.68,11.6,1.68,21.6,4.32
2,FTN1365232946SM,738,245000.0,171.6,32.0,2022-01-01 02:32:04,160.0,29.68,11.6,29.68,11.6,2.32
3,FTN1365211429IS,738,144500.0,131.6,24.0,2022-01-01 00:36:48,120.0,21.68,11.6,21.68,11.6,2.32
4,FTN1365211918AU,738,17000.0,51.6,8.0,2022-01-01 00:39:30,40.0,5.68,11.6,5.68,11.6,2.32


In [36]:
# precisely_chrgd calculation
df.loc[(df['CHARGE_DEVIATION'] == 0) & (df['TAX_DEVIATION'] == 0),'PRECISELY_CHARGED'] = True
df.loc[(df['CHARGE_DEVIATION'] != 0) | (df['TAX_DEVIATION'] != 0),'PRECISELY_CHARGED'] = False
df.head()

,TRANS_REFERENCE,TRANSACTION_CODE,TRANSACTION_AMOUNT,CHARGE_AMOUNT,TAX_AMOUNT,TIME_STAMP,EXPECTED_CHARGE,EXPECTED_TAX,CHARGE,TAX,CHARGE_DEVIATION,TAX_DEVIATION,PRECISELY_CHARGED
0,FTN1365221627DR,738,27500.0,71.6,12.0,2022-01-01 01:35:57,60.0,9.68,11.6,9.68,11.6,2.32,False
1,FTN1365214111SS,738,5000.0,41.6,6.0,2022-01-01 01:09:19,20.0,1.68,11.6,1.68,21.6,4.32,False
2,FTN1365232946SM,738,245000.0,171.6,32.0,2022-01-01 02:32:04,160.0,29.68,11.6,29.68,11.6,2.32,False
3,FTN1365211429IS,738,144500.0,131.6,24.0,2022-01-01 00:36:48,120.0,21.68,11.6,21.68,11.6,2.32,False
4,FTN1365211918AU,738,17000.0,51.6,8.0,2022-01-01 00:39:30,40.0,5.68,11.6,5.68,11.6,2.32,False


In [37]:
EXPECTED_COUNT = len(df['TRANS_REFERENCE'].unique().tolist())
PRECISELY_CHARGED = df['PRECISELY_CHARGED'].loc[df['PRECISELY_CHARGED'] == True].count()
NOT_PRECISELY_CHARGED = df['PRECISELY_CHARGED'].loc[df['PRECISELY_CHARGED'] == False].count()
MISSING_CHARGE = df['CHARGE_AMOUNT'].loc[df['CHARGE_AMOUNT'] == 0].count()
MISSING_TAX_BUT_CHARGE_PRESENT = df.loc[(df['CHARGE_AMOUNT'] != 0) & (df['TAX_AMOUNT'] == 0)].CHARGE_AMOUNT.count()
WRONG_CHARGE = df.loc[(df['CHARGE_AMOUNT'] != 0) & (df['CHARGE_DEVIATION'] != 0)].CHARGE_AMOUNT.count()

In [38]:
EXPECTED_CHARGE_AMOUNT = df['EXPECTED_CHARGE'].sum()
EXPECTED_TAX_AMOUNT = df['EXPECTED_TAX'].sum()
ACTUAL_CHARGE_AMOUNT = df['CHARGE_AMOUNT'].sum()
ACTUAL_TAX_AMOUNT = df['TAX_AMOUNT'].sum()
REVENUE_LOSS_OR_GAIN_IN_CHARGE_AMOUNT = ACTUAL_CHARGE_AMOUNT - EXPECTED_CHARGE_AMOUNT
REVENUE_LOSS_OR_GAIN_IN_TAX_AMOUNT = ACTUAL_TAX_AMOUNT - EXPECTED_TAX_AMOUNT
REVENUE_BELOW_ACCEPTANCE_IN_CHARGE_AMOUNT = df.loc[(df['CHARGE_AMOUNT'] != 0) & (df['CHARGE_DEVIATION'] < 0) ==True].CHARGE_DEVIATION.sum()
REVENUE_ABOVE_ACCEPTANCE_IN_CHARGE_AMOUNT = df.loc[(df['CHARGE_AMOUNT'] != 0) & (df['CHARGE_DEVIATION'] > 0) ==True].CHARGE_DEVIATION.sum()

In [39]:
print ("EXPECTED CHARGES COUNT - ", EXPECTED_COUNT)
print ("PRECISELY CHARGED COUNT - ", PRECISELY_CHARGED)
print ("NOT PRECISELY CHARGED COUNT - ", NOT_PRECISELY_CHARGED)
# SUBSET ONE
print("MISSING CHARGE COUNT - ",MISSING_CHARGE)
print("MISSING TAX BUT CHARGE PRESENT COUNT - ",MISSING_TAX_BUT_CHARGE_PRESENT)
# SUBSET TWO
print("WRONG CHARGE COUNT- ",WRONG_CHARGE)

EXPECTED CHARGES COUNT -  2363
PRECISELY CHARGED COUNT -  32
NOT PRECISELY CHARGED COUNT -  2331
MISSING CHARGE COUNT -  45
MISSING TAX BUT CHARGE PRESENT COUNT -  61
WRONG CHARGE COUNT-  2318


In [40]:
print ("PRECISELY CHARGED PERCENTAGE - " , round(PRECISELY_CHARGED / EXPECTED_COUNT * 100,2).astype(str)+'%')
print ("NOT PRECISELY CHARGED PERCENTAGE - " , round(NOT_PRECISELY_CHARGED / EXPECTED_COUNT * 100,2).astype(str)+'%')
print ("MISSING CHARGE PERCENTAGE - " , round(MISSING_CHARGE / EXPECTED_COUNT * 100,2).astype(str)+'%')
print ("MISSING TAX BUT CHARGE PRESENT PERCENTAGE - " , round(MISSING_TAX_BUT_CHARGE_PRESENT / EXPECTED_COUNT * 100,2).astype(str)+'%')
print ("WRONG CHARGE PERCENTAGE - " , round(WRONG_CHARGE / EXPECTED_COUNT * 100,2).astype(str)+'%')

PRECISELY CHARGED PERCENTAGE -  1.35%
NOT PRECISELY CHARGED PERCENTAGE -  98.65%
MISSING CHARGE PERCENTAGE -  1.9%
MISSING TAX BUT CHARGE PRESENT PERCENTAGE -  2.58%
WRONG CHARGE PERCENTAGE -  98.1%


In [41]:
print("EXPECTED CHARGE AMOUNT - ",round(EXPECTED_CHARGE_AMOUNT,2))
print("EXPECTED TAX AMOUNT - ",round(EXPECTED_TAX_AMOUNT,2))
print("ACTUAL CHARGE AMOUNT - ",round(ACTUAL_CHARGE_AMOUNT,2))
print("ACTUAL TAX AMOUNT - ",round(ACTUAL_TAX_AMOUNT,2))
print("REVENUE LOSS/EXTRA IN CHARGE AMOUNT - ",round(REVENUE_LOSS_OR_GAIN_IN_CHARGE_AMOUNT,2))
print("REVENUE LOSS/EXTRA IN TAX AMOUNT - ",round(REVENUE_LOSS_OR_GAIN_IN_TAX_AMOUNT,2))
print("REVENUE BELOW ACCEPTANCE IN CHARGE AMOUNT - ",round(REVENUE_BELOW_ACCEPTANCE_IN_CHARGE_AMOUNT,2))
print("REVENUE ABOVE ACCEPTANCE IN CHARGE AMOUNT - ",round(REVENUE_ABOVE_ACCEPTANCE_IN_CHARGE_AMOUNT,2))

EXPECTED CHARGE AMOUNT -  210850.0
EXPECTED TAX AMOUNT -  36762.08
ACTUAL CHARGE AMOUNT -  234578.8
ACTUAL TAX AMOUNT -  41630.0
REVENUE LOSS/EXTRA IN CHARGE AMOUNT -  23728.8
REVENUE LOSS/EXTRA IN TAX AMOUNT -  4867.92
REVENUE BELOW ACCEPTANCE IN CHARGE AMOUNT -  -2742.4
REVENUE ABOVE ACCEPTANCE IN CHARGE AMOUNT -  27101.2


In [42]:
print("ACTUAL CHARGE AMOUNT PERCENTAGE - ",round(ACTUAL_CHARGE_AMOUNT/EXPECTED_CHARGE_AMOUNT*100,2).astype(str)+'%')
print("ACTUAL TAX AMOUNT PERCENTAGE - ",round(ACTUAL_TAX_AMOUNT/EXPECTED_TAX_AMOUNT*100 ,2).astype(str)+'%')
print("REVENUE LOSS/EXTRA IN CHARGE AMOUNT PERCENTAGE - ",round(REVENUE_LOSS_OR_GAIN_IN_CHARGE_AMOUNT/EXPECTED_CHARGE_AMOUNT*100,2).astype(str)+'%')
print("REVENUE LOSS/EXTRA IN TAX AMOUNT PERCENTAGE - ",round(REVENUE_LOSS_OR_GAIN_IN_TAX_AMOUNT/EXPECTED_TAX_AMOUNT*100,2).astype(str)+'%')
print("REVENUE BELOW ACCEPTANCE IN CHARGE AMOUNT PERCENTAGE - ",round(REVENUE_BELOW_ACCEPTANCE_IN_CHARGE_AMOUNT/EXPECTED_CHARGE_AMOUNT*100,2).astype(str)+'%')
print("REVENUE ABOVE ACCEPTANCE IN CHARGE AMOUNT PERCENTAGE - ",round(REVENUE_ABOVE_ACCEPTANCE_IN_CHARGE_AMOUNT/EXPECTED_CHARGE_AMOUNT*100,2).astype(str)+'%')

ACTUAL CHARGE AMOUNT PERCENTAGE -  111.25%
ACTUAL TAX AMOUNT PERCENTAGE -  113.24%
REVENUE LOSS/EXTRA IN CHARGE AMOUNT PERCENTAGE -  11.25%
REVENUE LOSS/EXTRA IN TAX AMOUNT PERCENTAGE -  13.24%
REVENUE BELOW ACCEPTANCE IN CHARGE AMOUNT PERCENTAGE -  -1.3%
REVENUE ABOVE ACCEPTANCE IN CHARGE AMOUNT PERCENTAGE -  12.85%


In [43]:
split= "^"

EXPECTED_CHARGES = split.join(df['TRANS_REFERENCE'].unique().tolist())
PRECISELY_CHARGED = split.join(df.loc[df['PRECISELY_CHARGED'] == True].TRANS_REFERENCE.tolist())
NOT_PRECISELY_CHARGED = split.join(df.loc[df['PRECISELY_CHARGED'] == False].TRANS_REFERENCE.tolist())
MISSING_CHARGE = split.join(df.loc[df['CHARGE_AMOUNT'] == 0].TRANS_REFERENCE.tolist())
MISSING_TAX_BUT_CHARGE_PRESENT = split.join(df.loc[(df['CHARGE_AMOUNT'] != 0) & (df['TAX_AMOUNT'] == 0)].TRANS_REFERENCE.tolist())
WRONG_CHARGE = split.join(df.loc[(df['CHARGE_AMOUNT'] != 0) & (df['CHARGE_DEVIATION'] != 0)].TRANS_REFERENCE.tolist())
REVENUE_LOSS_EXTRA_IN_CHARGE = split.join(df.loc[df['CHARGE_DEVIATION'] != 0].TRANS_REFERENCE.tolist())
REVENUE_LOSS_EXTRA_IN_TAX = split.join(df.loc[df['TAX_DEVIATION'] == 0].TRANS_REFERENCE.tolist())
REVENUE_BELOW_ACCEPTANCE_IN_CHARGE = split.join(df.loc[(df['CHARGE_AMOUNT'] != 0) & (df['CHARGE_DEVIATION'] < 0) ==True].TRANS_REFERENCE.tolist())
REVENUE_ABOVE_ACCEPTANCE_IN_CHARGE = split.join(df.loc[(df['CHARGE_AMOUNT'] != 0) & (df['CHARGE_DEVIATION'] > 0) ==True].TRANS_REFERENCE.tolist())

In [44]:
from datetime import datetime
mysql_connection = mysql.connector.connect(**config)
def mysql_insertData(query,data):
    
    connection = mysql_connection
    cursor = connection.cursor()
    cursor.execute(query, data)
    connection.commit()
    primary_key = cursor.lastrowid
    cursor.close()
    connection.close()
    
    return primary_key

In [45]:
TRANSACTION_TYPE = 'MOBILE-EFT-91'
TRANSACTION_CODE = '186'
YEAR = '2013'
MONTH = '02'

param = {
            'TRANSACTION_TYPE' : TRANSACTION_TYPE,
            'TRANSACTION_CODE' : TRANSACTION_CODE,
            'YEAR' : YEAR,
            'MONTH' : MONTH,
            'EXPECTED_CHARGES' : EXPECTED_CHARGES,
            'PRECISELY_CHARGED' : PRECISELY_CHARGED,
            'NOT_PRECISELY_CHARGED' : NOT_PRECISELY_CHARGED,
            'MISSING_CHARGE' : MISSING_CHARGE,
            'MISSING_TAX_BUT_CHARGE_PRESENT' : MISSING_TAX_BUT_CHARGE_PRESENT,
            'WRONG_CHARGE' : WRONG_CHARGE,
            'REVENUE_LOSS_EXTRA_IN_CHARGE' : REVENUE_LOSS_EXTRA_IN_CHARGE,
            'REVENUE_LOSS_EXTRA_IN_TAX' : REVENUE_LOSS_EXTRA_IN_TAX,
            'REVENUE_BELOW_ACCEPTANCE_IN_CHARGE' : REVENUE_BELOW_ACCEPTANCE_IN_CHARGE,
            'REVENUE_ABOVE_ACCEPTANCE_IN_CHARGE' : REVENUE_ABOVE_ACCEPTANCE_IN_CHARGE,
            "CREATED_BY":'ML_USER',
            "CREATED_ON":datetime.now()
            
}

MYSQL_INSERT_TRANS_REFERENCE_DETAILS = ("INSERT INTO transaction$trans_reference$details"
                                        "(TRANSACTION_TYPE,TRANSACTION_CODE,YEAR,MONTH,EXPECTED_CHARGES,PRECISELY_CHARGED,NOT_PRECISELY_CHARGED,MISSING_CHARGE,MISSING_TAX_BUT_CHARGE_PRESENT,WRONG_CHARGE,REVENUE_LOSS_EXTRA_IN_CHARGE,REVENUE_LOSS_EXTRA_IN_TAX,REVENUE_BELOW_ACCEPTANCE_IN_CHARGE,REVENUE_ABOVE_ACCEPTANCE_IN_CHARGE,CREATED_BY,CREATED_ON)"
                                        "VALUES (%(TRANSACTION_TYPE)s, %(TRANSACTION_CODE)s,%(YEAR)s,%(MONTH)s, %(EXPECTED_CHARGES)s,%(PRECISELY_CHARGED)s,%(NOT_PRECISELY_CHARGED)s,%(MISSING_CHARGE)s,%(MISSING_TAX_BUT_CHARGE_PRESENT)s,%(WRONG_CHARGE)s,%(REVENUE_LOSS_EXTRA_IN_CHARGE)s,%(REVENUE_LOSS_EXTRA_IN_TAX)s,%(REVENUE_BELOW_ACCEPTANCE_IN_CHARGE)s,%(REVENUE_ABOVE_ACCEPTANCE_IN_CHARGE)s,%(CREATED_BY)s,%(CREATED_ON)s)")

mysql_insertData(MYSQL_INSERT_TRANS_REFERENCE_DETAILS,param)

14

In [46]:
def mysql_readTable(query,param=None):
    
    connection = mysql_connection
    dataframe = pd.read_sql(query, con=connection,params=param)
    
    return dataframe

In [47]:
import calendar

num_days = calendar.monthrange(int(YEAR), int(MONTH))[1]
START_DATE = YEAR + '-' + MONTH + '-' + '01'
END_DATE = YEAR + '-' + MONTH + '-' + str(num_days)

In [48]:
mysql_connection = mysql.connector.connect(**config)
param = {
            'TRAN_CODE':TRAN_CODE
        }
query = "select CHARGE_CODE,TAX from transaction$charge$codes where TRANSACTION_CODE = %(TRAN_CODE)s"
TRAN_CHARGE_CODES = mysql_readTable(query,param)
TRAN_CHARGE_CODES.head()
# TRAN_CHARGE_HISTORY['TRANSACTION_CODE'] = TRAN_CHARGE_HISTORY_1['TRANSACTION_CODE'][0]
# TRAN_CHARGE_HISTORY['TAX'] = TRAN_CHARGE_HISTORY_1['TAX'][0]
# TRAN_CHARGE_HISTORY.head()

,CHARGE_CODE,TAX
0,KITSCOMM,20
1,PESALINKCHG,None


In [49]:
id = TRAN_CHARGE_CODES['CHARGE_CODE'].unique().tolist()
FILTER_CODES = '|'.join(id)
param = {
            'CRG_CODE': FILTER_CODES
        }

In [50]:
# selecting the transaction code that should be charged
query = "select id,FLAT_AMOUNT,CALC_TYPE,PERCENTAGE,UPTO_AMOUNT,MINIMUM_AMOUNT,MAXIMUM_AMOUNT,DATE_TIME from bnk_charge$types where id REGEXP %(CRG_CODE)s"
query_his = "select id,FLAT_AMOUNT,CALC_TYPE,PERCENTAGE,UPTO_AMOUNT,MINIMUM_AMOUNT,MAXIMUM_AMOUNT,DATE_TIME from bnk_charge$types$history where id REGEXP %(CRG_CODE)s"
TRAN_CHARGE_HISTORY = mysql_readTable(query,param)
TRAN_CHARGE_HISTORY_1 = mysql_readTable(query_his,param)
TRAN_CHARGE_HISTORY = pd.concat([TRAN_CHARGE_HISTORY,TRAN_CHARGE_HISTORY_1], ignore_index=True)
TRAN_CHARGE_HISTORY = TRAN_CHARGE_HISTORY.sort_values(by='DATE_TIME', ascending=True)
TRAN_CHARGE_HISTORY.reset_index(drop=True, inplace=True)
TRAN_CHARGE_HISTORY['TRANSACTION_CODE'] = TRAN_CODE
TRAN_CHARGE_HISTORY['TAX'] = TRAN_CHARGE_CODES['TAX'][0]
TRAN_CHARGE_HISTORY.head()

,id,FLAT_AMOUNT,CALC_TYPE,PERCENTAGE,UPTO_AMOUNT,MINIMUM_AMOUNT,MAXIMUM_AMOUNT,DATE_TIME,TRANSACTION_CODE,TAX
0,KITSCOMM;1,None,LEVEL^LEVEL^LEVEL^LEVEL^LEVEL^LEVEL^LEVEL^LEVE...,0.00001^0.00001^0.00001^0.00001^0.00001^0.0000...,501.00^5001.00^10001.00^20001.00^50001.00^1000...,0.00^18.40^118.40^128.40^148.40^168.40^208.40^...,None,2022-01-01 12:39:00.0,738,20
1,KITSCOMM;2,None,LEVEL^LEVEL^LEVEL^LEVEL^LEVEL^LEVEL^LEVEL^LEVE...,0.00001^0.00001^0.00001^0.00001^0.00001^0.0000...,501.00^5001.00^10001.00^20001.00^50001.00^1000...,0.00^28.40^218.40^228.40^248.40^268.40^308.40^...,None,2022-01-10 12:39:00.0,738,20
2,KITSCOMM,None,LEVEL^LEVEL^LEVEL^LEVEL^LEVEL^LEVEL^LEVEL^LEVE...,0.00001^0.00001^0.00001^0.00001^0.00001^0.0000...,501.00^5001.00^10001.00^20001.00^50001.00^1000...,0.00^8.40^18.40^28.40^48.40^68.40^108.40^148.4...,None,None,738,20
3,PESALINKCHG,None,LEVEL^LEVEL,0.00001^0.00001,^501.00,0^11.60,None,None,738,20


In [51]:
TRAN_CHARGE_HISTORY['START_DATE'] = np.nan
TRAN_CHARGE_HISTORY['END_DATE'] = np.nan
for i in range (0,len(TRAN_CHARGE_HISTORY)):
    if (i ==0):
        TRAN_CHARGE_HISTORY['END_DATE'][i] = TRAN_CHARGE_HISTORY['DATE_TIME'][i]
    elif (TRAN_CHARGE_HISTORY['DATE_TIME'][i] == None):
        TRAN_CHARGE_HISTORY['START_DATE'][i] = str(pd.to_datetime(TRAN_CHARGE_HISTORY['DATE_TIME']).max())
    else:
        TRAN_CHARGE_HISTORY['START_DATE'][i] = TRAN_CHARGE_HISTORY['DATE_TIME'][i - 1]
        TRAN_CHARGE_HISTORY['END_DATE'][i] = TRAN_CHARGE_HISTORY['DATE_TIME'][i]
TRAN_CHARGE_HISTORY = TRAN_CHARGE_HISTORY[['id','FLAT_AMOUNT','CALC_TYPE','PERCENTAGE','UPTO_AMOUNT','MINIMUM_AMOUNT','MAXIMUM_AMOUNT','TRANSACTION_CODE','TAX','START_DATE','END_DATE']]
TRAN_CHARGE_HISTORY.head()

/tmp/ipykernel_51767/3791345217.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  TRAN_CHARGE_HISTORY['END_DATE'][i] = TRAN_CHARGE_HISTORY['DATE_TIME'][i]
/home/sabarish/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/tmp/ipykernel_51767/3791345217.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  TRAN_CHARGE_HISTORY['ST

,id,FLAT_AMOUNT,CALC_TYPE,PERCENTAGE,UPTO_AMOUNT,MINIMUM_AMOUNT,MAXIMUM_AMOUNT,TRANSACTION_CODE,TAX,START_DATE,END_DATE
0,KITSCOMM;1,None,LEVEL^LEVEL^LEVEL^LEVEL^LEVEL^LEVEL^LEVEL^LEVE...,0.00001^0.00001^0.00001^0.00001^0.00001^0.0000...,501.00^5001.00^10001.00^20001.00^50001.00^1000...,0.00^18.40^118.40^128.40^148.40^168.40^208.40^...,None,738,20,NaN,2022-01-01 12:39:00.0
1,KITSCOMM;2,None,LEVEL^LEVEL^LEVEL^LEVEL^LEVEL^LEVEL^LEVEL^LEVE...,0.00001^0.00001^0.00001^0.00001^0.00001^0.0000...,501.00^5001.00^10001.00^20001.00^50001.00^1000...,0.00^28.40^218.40^228.40^248.40^268.40^308.40^...,None,738,20,2022-01-01 12:39:00.0,2022-01-10 12:39:00.0
2,KITSCOMM,None,LEVEL^LEVEL^LEVEL^LEVEL^LEVEL^LEVEL^LEVEL^LEVE...,0.00001^0.00001^0.00001^0.00001^0.00001^0.0000...,501.00^5001.00^10001.00^20001.00^50001.00^1000...,0.00^8.40^18.40^28.40^48.40^68.40^108.40^148.4...,None,738,20,2022-01-10 12:39:00,NaN
3,PESALINKCHG,None,LEVEL^LEVEL,0.00001^0.00001,^501.00,0^11.60,None,738,20,2022-01-10 12:39:00,NaN


In [52]:
df_charges = TRAN_CHARGE_HISTORY
df_charges['START_DATE'] = pd.to_datetime(df_charges['START_DATE'])
df_charges['END_DATE'] = pd.to_datetime(df_charges['END_DATE'])
df['TIME_STAMP'] = pd.to_datetime(df['TIME_STAMP'])

df_charges.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   id                4 non-null      object        
 1   FLAT_AMOUNT       0 non-null      object        
 2   CALC_TYPE         4 non-null      object        
 3   PERCENTAGE        4 non-null      object        
 4   UPTO_AMOUNT       4 non-null      object        
 5   MINIMUM_AMOUNT    4 non-null      object        
 6   MAXIMUM_AMOUNT    0 non-null      object        
 7   TRANSACTION_CODE  4 non-null      object        
 8   TAX               4 non-null      object        
 9   START_DATE        3 non-null      datetime64[ns]
 10  END_DATE          2 non-null      datetime64[ns]
dtypes: datetime64[ns](2), object(9)
memory usage: 480.0+ bytes


In [53]:

if(df_charges['FLAT_AMOUNT'][0] == None):
    cols_to_df_list = ['CALC_TYPE','PERCENTAGE','UPTO_AMOUNT','MINIMUM_AMOUNT','MAXIMUM_AMOUNT','id','TAX']
    
    df_chg = pd.DataFrame(columns = cols_to_df_list)
    for j in range (0,len(df_charges)):
        df_len = len(df_chg)
        for i in cols_to_df_list:
            if df_charges[i][j]:
                split_list = fn_str_split(df_charges[i][j])
                for k in range (0,len(split_list)):
                    if((i == "id") | (i == 'TAX') | (i == 'START_DATE') | (i == 'END_DATE') ):
                        df_chg.loc[df_len + k,i] = df_charges[i][j]
                    else:
                        df_chg.loc[df_len + k,i] = split_list[k]
    
    
    df_chg['id'] = df_chg[['id']].ffill()
    df_chg['TAX'] = df_chg.groupby('id')['TAX'].ffill()
    df_chg=df_chg.replace(r'', 0, regex=True)
    
    
    
    df_chg['PERCENTAGE'] = df_chg['PERCENTAGE'].astype('float')
    df_chg['MAX_AMT'] = df_chg['UPTO_AMOUNT'].astype('float')
    df_chg['MIN_CHARGE'] = df_chg['MINIMUM_AMOUNT'].astype('float')
    df_chg['MAX_CHARGE'] = df_chg['MAXIMUM_AMOUNT'].astype('float')
    df_chg['TAX'] = df_chg['TAX'].astype('float')
    
    
    df_chg['MIN_AMT'] = 0
    for i in range(0,len(df_chg)):
        if(i == 0):
            df_chg['MIN_AMT'][i] =  0
        else:
            df_chg['MIN_AMT'][i] =  df_chg['MAX_AMT'][i-1] + 1

    df_chg.loc[(df_chg['MIN_AMT'].isnull()) | (df_chg['MIN_AMT'] > df_chg['MAX_AMT']),'MIN_AMT'] = 0
    df_chg = df_chg[["id","CALC_TYPE","PERCENTAGE","MIN_AMT","MAX_AMT","MIN_CHARGE","MAX_CHARGE","TAX"]]

df_chg['START_DATE'] = df_chg.id.map(df_charges.set_index('id')['START_DATE'])
df_chg['END_DATE'] = df_chg.id.map(df_charges.set_index('id')['END_DATE'])

df_chg

/tmp/ipykernel_51767/49456362.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_chg['MIN_AMT'][i] =  0
/tmp/ipykernel_51767/49456362.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_chg['MIN_AMT'][i] =  df_chg['MAX_AMT'][i-1] + 1
/home/sabarish/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,id,CALC_TYPE,PERCENTAGE,MIN_AMT,MAX_AMT,MIN_CHARGE,MAX_CHARGE,TAX,START_DATE,END_DATE
0,KITSCOMM;1,LEVEL,0.00001,0.0,501.0,0.0,NaN,20.0,NaT,2022-01-01 12:39:00
1,KITSCOMM;1,LEVEL,0.00001,502.0,5001.0,18.4,NaN,20.0,NaT,2022-01-01 12:39:00
2,KITSCOMM;1,LEVEL,0.00001,5002.0,10001.0,118.4,NaN,20.0,NaT,2022-01-01 12:39:00
3,KITSCOMM;1,LEVEL,0.00001,10002.0,20001.0,128.4,NaN,20.0,NaT,2022-01-01 12:39:00
4,KITSCOMM;1,LEVEL,0.00001,20002.0,50001.0,148.4,NaN,20.0,NaT,2022-01-01 12:39:00
5,KITSCOMM;1,LEVEL,0.00001,50002.0,100001.0,168.4,NaN,20.0,NaT,2022-01-01 12:39:00
6,KITSCOMM;1,LEVEL,0.00001,100002.0,200001.0,208.4,NaN,20.0,NaT,2022-01-01 12:39:00
7,KITSCOMM;1,LEVEL,0.00001,200002.0,500001.0,248.4,NaN,20.0,NaT,2022-01-01 12:39:00
8,KITSCOMM;1,LEVEL,0.00001,500002.0,NaN,288.4,NaN,20.0,NaT,2022-01-01 12:39:00
9,KITSCOMM;2,LEVEL,0.00001,0.0,501.0,0.0,NaN,20.0,2022-01-01 12:39:00,2022-01-10 12:39:00


In [54]:
df_chg.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29 entries, 0 to 28
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   id          29 non-null     object        
 1   CALC_TYPE   29 non-null     object        
 2   PERCENTAGE  29 non-null     float64       
 3   MIN_AMT     29 non-null     float64       
 4   MAX_AMT     26 non-null     float64       
 5   MIN_CHARGE  29 non-null     float64       
 6   MAX_CHARGE  0 non-null      float64       
 7   TAX         29 non-null     float64       
 8   START_DATE  20 non-null     datetime64[ns]
 9   END_DATE    18 non-null     datetime64[ns]
dtypes: datetime64[ns](2), float64(6), object(2)
memory usage: 3.5+ KB


In [55]:
df_chg.head()

,id,CALC_TYPE,PERCENTAGE,MIN_AMT,MAX_AMT,MIN_CHARGE,MAX_CHARGE,TAX,START_DATE,END_DATE
0,KITSCOMM;1,LEVEL,0.00001,0.0,501.0,0.0,NaN,20.0,NaT,2022-01-01 12:39:00
1,KITSCOMM;1,LEVEL,0.00001,502.0,5001.0,18.4,NaN,20.0,NaT,2022-01-01 12:39:00
2,KITSCOMM;1,LEVEL,0.00001,5002.0,10001.0,118.4,NaN,20.0,NaT,2022-01-01 12:39:00
3,KITSCOMM;1,LEVEL,0.00001,10002.0,20001.0,128.4,NaN,20.0,NaT,2022-01-01 12:39:00
4,KITSCOMM;1,LEVEL,0.00001,20002.0,50001.0,148.4,NaN,20.0,NaT,2022-01-01 12:39:00


In [56]:
df_charges

,id,FLAT_AMOUNT,CALC_TYPE,PERCENTAGE,UPTO_AMOUNT,MINIMUM_AMOUNT,MAXIMUM_AMOUNT,TRANSACTION_CODE,TAX,START_DATE,END_DATE
0,KITSCOMM;1,None,LEVEL^LEVEL^LEVEL^LEVEL^LEVEL^LEVEL^LEVEL^LEVE...,0.00001^0.00001^0.00001^0.00001^0.00001^0.0000...,501.00^5001.00^10001.00^20001.00^50001.00^1000...,0.00^18.40^118.40^128.40^148.40^168.40^208.40^...,None,738,20,NaT,2022-01-01 12:39:00
1,KITSCOMM;2,None,LEVEL^LEVEL^LEVEL^LEVEL^LEVEL^LEVEL^LEVEL^LEVE...,0.00001^0.00001^0.00001^0.00001^0.00001^0.0000...,501.00^5001.00^10001.00^20001.00^50001.00^1000...,0.00^28.40^218.40^228.40^248.40^268.40^308.40^...,None,738,20,2022-01-01 12:39:00,2022-01-10 12:39:00
2,KITSCOMM,None,LEVEL^LEVEL^LEVEL^LEVEL^LEVEL^LEVEL^LEVEL^LEVE...,0.00001^0.00001^0.00001^0.00001^0.00001^0.0000...,501.00^5001.00^10001.00^20001.00^50001.00^1000...,0.00^8.40^18.40^28.40^48.40^68.40^108.40^148.4...,None,738,20,2022-01-10 12:39:00,NaT
3,PESALINKCHG,None,LEVEL^LEVEL,0.00001^0.00001,^501.00,0^11.60,None,738,20,2022-01-10 12:39:00,NaT


In [58]:
# For FLAT AMOUNT with history
for i in range(0,len(df_charges)):
    if(df_charges['FLAT_AMOUNT'][i] != None):
        # Creating expected charge and expected tax percentage column
        if(pd.isnull(df_charges['START_DATE'][i])):
            df.loc[(df['TIME_STAMP'] <= df_charges['END_DATE'][i]),'EXPECTED_CHARGE'] = df_charges['FLAT_AMOUNT'][i]
            df.loc[(df['TIME_STAMP'] <= df_charges['END_DATE'][i]),'EXPECTED_TAX'] = df_charges['TAX'][i]
        elif(pd.isnull(df_charges['END_DATE'][i])):
            df.loc[(df['TIME_STAMP'] >= df_charges['START_DATE'][i]),'EXPECTED_CHARGE'] = df_charges['FLAT_AMOUNT'][i]
            df.loc[(df['TIME_STAMP'] >= df_charges['START_DATE'][i]),'EXPECTED_TAX'] = df_charges['TAX'][i]
        else:
            df.loc[(df['TIME_STAMP'] >= df_charges['START_DATE'][i]) & (df['TIME_STAMP'] <= df_charges['END_DATE'][i]),'EXPECTED_CHARGE'] = df_charges['FLAT_AMOUNT'][i]
            df.loc[(df['TIME_STAMP'] >= df_charges['START_DATE'][i]) & (df['TIME_STAMP'] <= df_charges['END_DATE'][i]),'EXPECTED_TAX'] = df_charges['TAX'][i]
        #conversion
        df['EXPECTED_CHARGE'] = df['EXPECTED_CHARGE'].astype('float')
        df['EXPECTED_TAX'] = df['EXPECTED_TAX'].astype('float')
        # expected tax calculation
        df['EXPECTED_TAX'] = df['EXPECTED_CHARGE'] /100 * df['EXPECTED_TAX']
    
    
    elif((df_charges['CALC_TYPE'][i] == 'LEVEL') | (df_charges['CALC_TYPE'][i] == 'BAND')): 
        if(pd.isnull(df_charges['START_DATE'][i])):
            df.loc[(df['TIME_STAMP'] <= df_charges['END_DATE'][i]),'EXPECTED_CHARGE'] = df_charges['PERCENTAGE'][i]
            df.loc[(df['TIME_STAMP'] <= df_charges['END_DATE'][i]),'EXPECTED_TAX'] = df_charges['TAX'][i]
        elif(pd.isnull(df_charges['END_DATE'][i])):
            df.loc[(df['TIME_STAMP'] >= df_charges['START_DATE'][i]),'EXPECTED_CHARGE'] = df_charges['PERCENTAGE'][i]
            df.loc[(df['TIME_STAMP'] >= df_charges['START_DATE'][i]),'EXPECTED_TAX'] = df_charges['TAX'][i]
        else:
            df.loc[(df['TIME_STAMP'] >= df_charges['START_DATE'][i]) & (df['TIME_STAMP'] <= df_charges['END_DATE'][i]),'EXPECTED_CHARGE'] = df_charges['PERCENTAGE'][i]
            df.loc[(df['TIME_STAMP'] >= df_charges['START_DATE'][i]) & (df['TIME_STAMP'] <= df_charges['END_DATE'][i]),'EXPECTED_TAX'] = df_charges['TAX'][i] 
        #conversion
        df['EXPECTED_CHARGE'] = df['EXPECTED_CHARGE'].astype('float')
        df['EXPECTED_TAX'] = df['EXPECTED_TAX'].astype('float')
        df_charges['MINIMUM_AMOUNT'] = df_charges['MINIMUM_AMOUNT'].astype('float')
        df_charges['MAXIMUM_AMOUNT'] = df_charges['MAXIMUM_AMOUNT'].astype('float')
        # expected charge and tax calculation
        df['EXPECTED_CHARGE'] = round(df['TRANSACTION_AMOUNT'] /100 * df['EXPECTED_CHARGE'],2)
        df['EXPECTED_TAX'] = round(df['EXPECTED_CHARGE'] /100 * df['EXPECTED_TAX'],2)
        # comparing charges with minimum maximum amount
        if(pd.isnull(df_charges['START_DATE'][i])):
            # for charge lesser then minimum charge amount
            if(pd.isnull(df_charges.iloc[i]['MINIMUM_AMOUNT']) == False):
                df.loc[(df['TIME_STAMP'] <= df_charges['END_DATE'][i]) & (df['EXPECTED_CHARGE'] < df_charges['MINIMUM_AMOUNT'][i]) , 'EXPECTED_CHARGE'] =  df_charges['MINIMUM_AMOUNT'][i]
            # for charge greter then minimum charge amount
            if(pd.isnull(df_charges.iloc[i]['MAXIMUM_AMOUNT']) == False):
                df.loc[(df['TIME_STAMP'] <= df_charges['END_DATE'][i]) & (df['EXPECTED_CHARGE'] > df_charges['MAXIMUM_AMOUNT'][i]) , 'EXPECTED_CHARGE'] =  df_charges['MAXIMUM_AMOUNT'][i]
        elif(pd.isnull(df_charges['END_DATE'][i])):
            # for charge lesser then minimum charge amount
            if(pd.isnull(df_charges.iloc[i]['MINIMUM_AMOUNT']) == False):
                df.loc[(df['TIME_STAMP'] >= df_charges['START_DATE'][i]) & (df['EXPECTED_CHARGE'] < df_charges['MINIMUM_AMOUNT'][i]) , 'EXPECTED_CHARGE'] =  df_charges['MINIMUM_AMOUNT'][i]
            # for charge greter then minimum charge amount
            if(pd.isnull(df_charges.iloc[i]['MAXIMUM_AMOUNT']) == False):
                df.loc[(df['TIME_STAMP'] >= df_charges['START_DATE'][i]) & (df['EXPECTED_CHARGE'] > df_charges['MAXIMUM_AMOUNT'][i]) , 'EXPECTED_CHARGE'] =  df_charges['MAXIMUM_AMOUNT'][i]
        else:
            # for charge lesser then minimum charge amount
            if(pd.isnull(df_charges.iloc[i]['MINIMUM_AMOUNT']) == False):
                df.loc[(df['TIME_STAMP'] >= df_charges['START_DATE'][i]) & (df['TIME_STAMP'] <= df_charges['END_DATE'][i]) & (df['EXPECTED_CHARGE'] < df_charges['MINIMUM_AMOUNT'][i]) , 'EXPECTED_CHARGE'] =  df_charges['MINIMUM_AMOUNT'][i]
            # for charge greter then minimum charge amount
            if(pd.isnull(df_charges.iloc[i]['MAXIMUM_AMOUNT']) == False):
                df.loc[(df['TIME_STAMP'] >= df_charges['START_DATE'][i]) & (df['TIME_STAMP'] <= df_charges['END_DATE'][i]) & (df['EXPECTED_CHARGE'] > df_charges['MAXIMUM_AMOUNT'][i]) , 'EXPECTED_CHARGE'] =  df_charges['MAXIMUM_AMOUNT'][i]

In [59]:
# for level charge with no hostory
i = 0
if(df_charges['FLAT_AMOUNT'][i] == None):
    if(df_chg['CALC_TYPE'][0] == "LEVEL"):
        # for expected charge initialization
        df['EXPECTED_CHARGE'] = 0
        df['EXPECTED_TAX'] = 0

        for i in range (0,len(df_chg)):
            if(pd.isnull(df_chg['START_DATE'][i])):
                # for charge as per percentage
                df.loc[(df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df['TRANSACTION_AMOUNT'] <= df_chg['MAX_AMT'][i]), 'CHARGE'] = round((df['TRANSACTION_AMOUNT'] * df_chg['PERCENTAGE'][i] /100),2)
                # for charge lesser then minimum charge amount
                if(pd.isnull(df_chg.iloc[i]['MIN_CHARGE']) == False):
                    df.loc[(df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df['TRANSACTION_AMOUNT'] <= df_chg['MAX_AMT'][i]) & (df['CHARGE'] < df_chg['MIN_CHARGE'][i]) , 'CHARGE'] =  df_chg['MIN_CHARGE'][i]
                # for charge greter then minimum charge amount
                if(pd.isnull(df_chg.iloc[i]['MAX_CHARGE']) == False):
                    df.loc[(df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df['TRANSACTION_AMOUNT'] <= df_chg['MAX_AMT'][i]) & (df['CHARGE'] > df_chg['MAX_CHARGE'][i]) , 'CHARGE'] =  df_chg['MAX_CHARGE'][i]
                # for tax as per percentage
                if(pd.isnull(df_chg.iloc[i]['TAX']) == False):
                    df.loc[(df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df['TRANSACTION_AMOUNT'] <= df_chg['MAX_AMT'][i]), 'TAX'] = round((df['CHARGE'] * df_chg['TAX'][i] /100),2)

                # for expected charge
                df.loc[(df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df['TRANSACTION_AMOUNT'] <= df_chg['MAX_AMT'][i]), 'EXPECTED_CHARGE'] = round((df['EXPECTED_CHARGE'] + df['CHARGE']),2)
                # for expected tax
                if(pd.isnull(df_chg.iloc[i]['TAX']) == False):
                    df.loc[(df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df['TRANSACTION_AMOUNT'] <= df_chg['MAX_AMT'][i]), 'EXPECTED_TAX'] = round((df['EXPECTED_TAX'] + df['TAX']),2)
                
                
            elif(pd.isnull(df_chg['END_DATE'][i])):  
                # for charge as per percentage
                df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df['TRANSACTION_AMOUNT'] <= df_chg['MAX_AMT'][i]), 'CHARGE'] = round((df['TRANSACTION_AMOUNT'] * df_chg['PERCENTAGE'][i] /100),2)
                # for charge lesser then minimum charge amount
                if(pd.isnull(df_chg.iloc[i]['MIN_CHARGE']) == False):
                    df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df['TRANSACTION_AMOUNT'] <= df_chg['MAX_AMT'][i]) & (df['CHARGE'] < df_chg['MIN_CHARGE'][i]) , 'CHARGE'] =  df_chg['MIN_CHARGE'][i]
                # for charge greter then minimum charge amount
                if(pd.isnull(df_chg.iloc[i]['MAX_CHARGE']) == False):
                    df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df['TRANSACTION_AMOUNT'] <= df_chg['MAX_AMT'][i]) & (df['CHARGE'] > df_chg['MAX_CHARGE'][i]) , 'CHARGE'] =  df_chg['MAX_CHARGE'][i]
                # for tax as per percentage
                if(pd.isnull(df_chg.iloc[i]['TAX']) == False):
                    df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df['TRANSACTION_AMOUNT'] <= df_chg['MAX_AMT'][i]), 'TAX'] = round((df['CHARGE'] * df_chg['TAX'][i] /100),2)

                # for expected charge
                df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df['TRANSACTION_AMOUNT'] <= df_chg['MAX_AMT'][i]), 'EXPECTED_CHARGE'] = round((df['EXPECTED_CHARGE'] + df['CHARGE']),2)
                # for expected tax & 
                if(pd.isnull(df_chg.iloc[i]['TAX']) == False):
                    df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df['TRANSACTION_AMOUNT'] <= df_chg['MAX_AMT'][i]), 'EXPECTED_TAX'] = round((df['EXPECTED_TAX'] + df['TAX']),2)

                    
                    
            else:
                # for charge as per percentage
                df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df['TRANSACTION_AMOUNT'] <= df_chg['MAX_AMT'][i]), 'CHARGE'] = round((df['TRANSACTION_AMOUNT'] * df_chg['PERCENTAGE'][i] /100),2)
                # for charge lesser then minimum charge amount
                if(pd.isnull(df_chg.iloc[i]['MIN_CHARGE']) == False):
                    df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df['TRANSACTION_AMOUNT'] <= df_chg['MAX_AMT'][i]) & (df['CHARGE'] < df_chg['MIN_CHARGE'][i]) , 'CHARGE'] =  df_chg['MIN_CHARGE'][i]
                # for charge greter then minimum charge amount
                if(pd.isnull(df_chg.iloc[i]['MAX_CHARGE']) == False):
                    df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df['TRANSACTION_AMOUNT'] <= df_chg['MAX_AMT'][i]) & (df['CHARGE'] > df_chg['MAX_CHARGE'][i]) , 'CHARGE'] =  df_chg['MAX_CHARGE'][i]
                # for tax as per percentage
                if(pd.isnull(df_chg.iloc[i]['TAX']) == False):
                    df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df['TRANSACTION_AMOUNT'] <= df_chg['MAX_AMT'][i]), 'TAX'] = round((df['CHARGE'] * df_chg['TAX'][i] /100),2)

                # for expected charge
                df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df['TRANSACTION_AMOUNT'] <= df_chg['MAX_AMT'][i]), 'EXPECTED_CHARGE'] = round((df['EXPECTED_CHARGE'] + df['CHARGE']),2)
                # for expected tax
                if(pd.isnull(df_chg.iloc[i]['TAX']) == False):
                    df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df['TRANSACTION_AMOUNT'] <= df_chg['MAX_AMT'][i]), 'EXPECTED_TAX'] = round((df['EXPECTED_TAX'] + df['TAX']),2)

                

        df_chg = df_chg.loc[(df_chg['MAX_AMT'].isnull())]
        df_chg.reset_index(drop=True, inplace=True)

        for i in range (0,len(df_chg)):
            if(pd.isnull(df_chg['START_DATE'][i])):
                # for charge as per percentage
                df.loc[(df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'CHARGE'] = round((df['TRANSACTION_AMOUNT'] * df_chg['PERCENTAGE'][i] /100),2)
                # for charge lesser then minimum charge amount
                if(pd.isnull(df_chg.iloc[i]['MIN_CHARGE']) == False):
                    df.loc[(df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df['CHARGE'] < df_chg['MIN_CHARGE'][i]) , 'CHARGE'] =  df_chg['MIN_CHARGE'][i]
                # for charge greter then minimum charge amount
                if(pd.isnull(df_chg.iloc[i]['MAX_CHARGE']) == False):
                    df.loc[(df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df['CHARGE'] > df_chg['MAX_CHARGE'][i]) , 'CHARGE'] =  df_chg['MAX_CHARGE'][i]
                # for tax as per percentage
                if(pd.isnull(df_chg.iloc[i]['TAX']) == False):
                    df.loc[(df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'TAX'] = round((df['CHARGE'] * df_chg['TAX'][i] /100),2)

                # for expected charge
                df.loc[(df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'EXPECTED_CHARGE'] = round((df['EXPECTED_CHARGE'] + df['CHARGE']),2)
                # for expected tax
                if(pd.isnull(df_chg.iloc[i]['TAX']) == False):
                    df.loc[(df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'EXPECTED_TAX'] = round((df['EXPECTED_TAX'] + df['TAX']),2)  
            
            
            elif(pd.isnull(df_chg['END_DATE'][i])):
                # for charge as per percentage
                df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'CHARGE'] = round((df['TRANSACTION_AMOUNT'] * df_chg['PERCENTAGE'][i] /100),2)
                # for charge lesser then minimum charge amount
                if(pd.isnull(df_chg.iloc[i]['MIN_CHARGE']) == False):
                    df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df['CHARGE'] < df_chg['MIN_CHARGE'][i]) , 'CHARGE'] =  df_chg['MIN_CHARGE'][i]
                # for charge greter then minimum charge amount
                if(pd.isnull(df_chg.iloc[i]['MAX_CHARGE']) == False):
                    df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df['CHARGE'] > df_chg['MAX_CHARGE'][i]) , 'CHARGE'] =  df_chg['MAX_CHARGE'][i]
                # for tax as per percentage
                if(pd.isnull(df_chg.iloc[i]['TAX']) == False):
                    df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'TAX'] = round((df['CHARGE'] * df_chg['TAX'][i] /100),2)

                # for expected charge
                df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'EXPECTED_CHARGE'] = round((df['EXPECTED_CHARGE'] + df['CHARGE']),2)
                # for expected tax
                if(pd.isnull(df_chg.iloc[i]['TAX']) == False):
                    df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'EXPECTED_TAX'] = round((df['EXPECTED_TAX'] + df['TAX']),2)  
            
            
            else:
                # for charge as per percentage
                df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'CHARGE'] = round((df['TRANSACTION_AMOUNT'] * df_chg['PERCENTAGE'][i] /100),2)
                # for charge lesser then minimum charge amount
                if(pd.isnull(df_chg.iloc[i]['MIN_CHARGE']) == False):
                    df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df['CHARGE'] < df_chg['MIN_CHARGE'][i]) , 'CHARGE'] =  df_chg['MIN_CHARGE'][i]
                # for charge greter then minimum charge amount
                if(pd.isnull(df_chg.iloc[i]['MAX_CHARGE']) == False):
                    df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df['CHARGE'] > df_chg['MAX_CHARGE'][i]) , 'CHARGE'] =  df_chg['MAX_CHARGE'][i]
                # for tax as per percentage
                if(pd.isnull(df_chg.iloc[i]['TAX']) == False):
                    df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'TAX'] = round((df['CHARGE'] * df_chg['TAX'][i] /100),2)

                # for expected charge
                df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'EXPECTED_CHARGE'] = round((df['EXPECTED_CHARGE'] + df['CHARGE']),2)
                # for expected tax
                if(pd.isnull(df_chg.iloc[i]['TAX']) == False):
                    df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'EXPECTED_TAX'] = round((df['EXPECTED_TAX'] + df['TAX']),2)  

In [60]:
# for band charge with hostory
i = 0
if(df_charges['FLAT_AMOUNT'][i] == None):
    if(df_chg['CALC_TYPE'][0] == "BAND"): 
        # for expected charge initialization
        df['EXPECTED_CHARGE'] = 0
        df['EXPECTED_TAX'] = 0

        for i in range (0,len(df_chg)):
            if(pd.isnull(df_chg['START_DATE'][i])):
                # Chargable amount calculation
                df.loc[(df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'CHARGABLE_AMT'] = df['TRANSACTION_AMOUNT'] - (df_chg['MIN_AMT'][i])
                if(pd.isnull(df_chg.iloc[i]['MAX_AMT']) == False):
                    df.loc[(df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['CHARGABLE_AMT'] > (df_chg['MAX_AMT'][i] - df_chg['MIN_AMT'][i])), 'CHARGABLE_AMT'] = df_chg['MAX_AMT'][i] - df_chg['MIN_AMT'][i]
                else:
                    df.loc[(df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] > (df_chg['MIN_AMT'][i])), 'CHARGABLE_AMT'] = df['TRANSACTION_AMOUNT'] - (df_chg['MIN_AMT'][i])
                # for charge as per percentage
                df.loc[(df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'CHARGE'] = round((df['CHARGABLE_AMT'] * df_chg['PERCENTAGE'][i] /100),2)
                # for tax as per percentage
                if(pd.isnull(df_chg.iloc[i]['TAX']) == False):
                    df.loc[(df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'TAX'] = round((df['CHARGE'] * df_chg['TAX'][i] /100),2)

                # for expected charge
                df.loc[(df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'EXPECTED_CHARGE'] = round((df['EXPECTED_CHARGE'] + df['CHARGE']),2)
                # for expected tax
                if(pd.isnull(df_chg.iloc[i]['TAX']) == False):
                    df.loc[(df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'EXPECTED_TAX'] = round((df['EXPECTED_TAX'] + df['TAX']),2)
                
            
            elif(pd.isnull(df_chg['END_DATE'][i])):
                # Chargable amount calculation
                df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'CHARGABLE_AMT'] = df['TRANSACTION_AMOUNT'] - (df_chg['MIN_AMT'][i])
                if(pd.isnull(df_chg.iloc[i]['MAX_AMT']) == False):
                    df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['CHARGABLE_AMT'] > (df_chg['MAX_AMT'][i] - df_chg['MIN_AMT'][i])), 'CHARGABLE_AMT'] = df_chg['MAX_AMT'][i] - df_chg['MIN_AMT'][i]
                else:
                    df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TRANSACTION_AMOUNT'] > (df_chg['MIN_AMT'][i])), 'CHARGABLE_AMT'] = df['TRANSACTION_AMOUNT'] - (df_chg['MIN_AMT'][i])
                # for charge as per percentage
                df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'CHARGE'] = round((df['CHARGABLE_AMT'] * df_chg['PERCENTAGE'][i] /100),2)
                # for tax as per percentage
                if(pd.isnull(df_chg.iloc[i]['TAX']) == False):
                    df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'TAX'] = round((df['CHARGE'] * df_chg['TAX'][i] /100),2)

                # for expected charge
                df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'EXPECTED_CHARGE'] = round((df['EXPECTED_CHARGE'] + df['CHARGE']),2)
                # for expected tax
                if(pd.isnull(df_chg.iloc[i]['TAX']) == False):
                    df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'EXPECTED_TAX'] = round((df['EXPECTED_TAX'] + df['TAX']),2)
                
            
            else:
                # Chargable amount calculation
                df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'CHARGABLE_AMT'] = df['TRANSACTION_AMOUNT'] - (df_chg['MIN_AMT'][i])
                if(pd.isnull(df_chg.iloc[i]['MAX_AMT']) == False):
                    df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['CHARGABLE_AMT'] > (df_chg['MAX_AMT'][i] - df_chg['MIN_AMT'][i])), 'CHARGABLE_AMT'] = df_chg['MAX_AMT'][i] - df_chg['MIN_AMT'][i]
                else:
                    df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] > (df_chg['MIN_AMT'][i])), 'CHARGABLE_AMT'] = df['TRANSACTION_AMOUNT'] - (df_chg['MIN_AMT'][i])
                # for charge as per percentage
                df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'CHARGE'] = round((df['CHARGABLE_AMT'] * df_chg['PERCENTAGE'][i] /100),2)
                # for tax as per percentage
                if(pd.isnull(df_chg.iloc[i]['TAX']) == False):
                    df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'TAX'] = round((df['CHARGE'] * df_chg['TAX'][i] /100),2)

                # for expected charge
                df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'EXPECTED_CHARGE'] = round((df['EXPECTED_CHARGE'] + df['CHARGE']),2)
                # for expected tax
                if(pd.isnull(df_chg.iloc[i]['TAX']) == False):
                    df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'EXPECTED_TAX'] = round((df['EXPECTED_TAX'] + df['TAX']),2)

In [ ]:
# Need's to handle
#--------------------
# LEVEL
# BAND
# MULTIPLE LEVELS/MULTIPLE BANDS
# DIFFERENT CURRENCY
# DIFFERENT CURRENCY WITH LEVELS/BANDS
# PERFORMANCE
#-----------------
# MONTH WISE EXECUTION
# DAILY DATA STORAGE IN IN DB 

In [ ]:
# import numpy as np
# from itertools import chain

In [ ]:
# # selecting the transaction code charges
# mysql_connection = mysql.connector.connect(**config)
# #df_charges = pd.read_sql('select tcc.TRANSACTION_CODE,bct.CALCULATION_BASIS,bct.FLAT_AMOUNT,bct.CALC_TYPE,bct.PERCENTAGE,bct.UPTO_AMOUNT,bct.MAXIMUM_AMOUNT,bct.MAXIMUM_AMOUNT,tcc.TAX from bnk_charge$types bct inner join transaction$charge$codes tcc on bct.id = tcc.CHARGE_CODE where tcc.TRANSACTION_CODE = 738', con=mysql_connection)
# df_charges = pd.read_sql('select * from  bnk_charge$types where id ="TEST"', con=mysql_connection)
# mysql_connection.close()
# df_charges.head()

In [ ]:
# # return list from series of comma-separated strings
# def chainer(s):
#     return list(chain.from_iterable(s.str.split('^')))

In [ ]:
# # calculate lengths of splits
# lens = df_charges['CALC_TYPE'].str.split('^').map(len)

# # create new dataframe, repeating or chaining as appropriate
# df_charges = pd.DataFrame({'TRANSACTION_CODE': np.repeat(df_charges['id'], lens),
#                     'CALCULATION_BASIS':np.repeat(df_charges['CALCULATION_BASIS'],lens),
#                     'CALC_TYPE': chainer(df_charges['CALC_TYPE']),
#                     'PERCENTAGE': chainer(df_charges['PERCENTAGE']),
#                     'MAX_AMT': chainer(df_charges['UPTO_AMOUNT']),
#                     'MIN_CHARGE': chainer(df_charges['MINIMUM_AMOUNT']),
#                     'MAX_CHARGE': chainer(df_charges['MAXIMUM_AMOUNT'])})

# df_charges.reset_index(drop=True, inplace=True)
# df_charges

In [ ]:
# df_charges['PERCENTAGE'] = df_charges['PERCENTAGE'].astype('float')
# df_charges['MAX_AMT'] = df_charges['MAX_AMT'].astype('int')
# df_charges['MIN_CHARGE'] = df_charges['MIN_CHARGE'].astype('int')
# df_charges['MAX_CHARGE'] = df_charges['MAX_CHARGE'].astype('int')

In [ ]:
# df_charges['MIN_AMT'] = 0
# for i in range(0,len(df_charges)):
#     if(i == 0):
#         df_charges['MIN_AMT'][i] =  0
#     else:
#         df_charges['MIN_AMT'][i] =  df_charges['MAX_AMT'][i-1] + 1

# df_charges = df_charges[["TRANSACTION_CODE","CALCULATION_BASIS","CALC_TYPE","PERCENTAGE","MIN_AMT","MAX_AMT","MIN_CHARGE","MAX_CHARGE"]]
# df_charges

In [ ]:
# # fdr level calculations
# df1 = pd.read_csv("sample.csv")
# df1.head()

In [ ]:
# # for expected charge initialization
# df1['EXPECTED_CHARGE'] = 0

# for i in range (0,len(df_charges)):
    
#     # for expected charge as per percentage
#     df1.loc[(df1['TRANSACTION_AMOUNT'] >= df_charges['MIN_AMT'][i]) & (df1['TRANSACTION_AMOUNT'] <= df_charges['MAX_AMT'][i]), 'EXPECTED_CHARGE'] = round(df1['EXPECTED_CHARGE'] + (df1['TRANSACTION_AMOUNT'] * df_charges['PERCENTAGE'][i] /100),2)
#     # for expected charge lesser then minimum charge amount
#     df1.loc[(df1['TRANSACTION_AMOUNT'] >= df_charges['MIN_AMT'][i]) & (df1['TRANSACTION_AMOUNT'] <= df_charges['MAX_AMT'][i]) & (df1['EXPECTED_CHARGE'] < df_charges['MIN_CHARGE'][i]) , 'EXPECTED_CHARGE'] =  df_charges['MIN_CHARGE'][i]
#     # for expected charge greter then minimum charge amount
#     df1.loc[(df1['TRANSACTION_AMOUNT'] >= df_charges['MIN_AMT'][i]) & (df1['TRANSACTION_AMOUNT'] <= df_charges['MAX_AMT'][i]) & (df1['EXPECTED_CHARGE'] > df_charges['MAX_CHARGE'][i]) , 'EXPECTED_CHARGE'] =  df_charges['MAX_CHARGE'][i]

# # df_charges.loc[df_charges['MAX_AMT'].idxmax()]
# # for mapping percentage of charge, greater then last upto amount
# df1.loc[(df1['TRANSACTION_AMOUNT'] > df_charges.loc[df_charges['MAX_AMT'].idxmax()].MAX_AMT),'EXPECTED_CHARGE'] = round(df1['EXPECTED_CHARGE'] + (df1['TRANSACTION_AMOUNT'] * df_charges.loc[df_charges['MAX_AMT'].idxmax()].PERCENTAGE /100),2)
# # for mapping min charge, greater then last upto amount
# df1.loc[(df1['TRANSACTION_AMOUNT'] > df_charges.loc[df_charges['MAX_AMT'].idxmax()].MAX_AMT) & (df1['EXPECTED_CHARGE'] < df_charges.loc[df_charges['MAX_AMT'].idxmax()].MIN_CHARGE),'EXPECTED_CHARGE'] = df_charges.loc[df_charges['MAX_AMT'].idxmax()].MIN_CHARGE
# # for mapping max charge, greater then last upto amount
# df1.loc[(df1['TRANSACTION_AMOUNT'] > df_charges.loc[df_charges['MAX_AMT'].idxmax()].MAX_AMT) & (df1['EXPECTED_CHARGE'] > df_charges.loc[df_charges['MAX_AMT'].idxmax()].MAX_CHARGE),'EXPECTED_CHARGE'] = df_charges.loc[df_charges['MAX_AMT'].idxmax()].MAX_CHARGE
# df1.head()

In [ ]:
# # for band calculations
# df1 = pd.read_csv("sample.csv")
# df1.head()

In [ ]:
# # for expected charge initialization
# df1['EXPECTED_CHARGE'] = 0

# for i in range (0,len(df_charges)):
#     # Chargable amount calculation
#     df1.loc[(df1['TRANSACTION_AMOUNT'] >= df_charges['MIN_AMT'][i]), 'CHARGABLE_AMT'] = df1['TRANSACTION_AMOUNT'] - (df_charges['MIN_AMT'][i])
#     df1.loc[(df1['CHARGABLE_AMT'] > (df_charges['MAX_AMT'][i] - df_charges['MIN_AMT'][i])), 'CHARGABLE_AMT'] = df_charges['MAX_AMT'][i] - df_charges['MIN_AMT'][i]
#     # for expected charge as per percentage
#     df1.loc[(df1['TRANSACTION_AMOUNT'] >= df_charges['MIN_AMT'][i]), 'EXPECTED_CHARGE'] = round(df1['EXPECTED_CHARGE'] + (df1['CHARGABLE_AMT'] * df_charges['PERCENTAGE'][i] /100),2)

# # for mapping percentage of charge, greater then last upto amount
# df1.loc[(df1['TRANSACTION_AMOUNT'] > df_charges.loc[df_charges['MAX_AMT'].idxmax()].MAX_AMT),'CHARGABLE_AMT'] = df1['TRANSACTION_AMOUNT'] - df_charges.loc[df_charges['MAX_AMT'].idxmax()].MAX_AMT
# # for expected charge as per last percentage
# df1.loc[(df1['TRANSACTION_AMOUNT'] > df_charges.loc[df_charges['MAX_AMT'].idxmax()].MAX_AMT), 'EXPECTED_CHARGE'] = round(df1['EXPECTED_CHARGE'] + (df1['CHARGABLE_AMT'] * df_charges.loc[df_charges['MAX_AMT'].idxmax()].PERCENTAGE /100),2)
# df1.head()

In [ ]:
# def fn_str_split(string):
#     return string.split('^')

In [ ]:
# # selecting the transaction code charges
# mysql_connection = mysql.connector.connect(**config)
# #df_charges = pd.read_sql('select tcc.TRANSACTION_CODE,bct.CALCULATION_BASIS,bct.FLAT_AMOUNT,bct.CALC_TYPE,bct.PERCENTAGE,bct.UPTO_AMOUNT,bct.MAXIMUM_AMOUNT,bct.MAXIMUM_AMOUNT,tcc.TAX from bnk_charge$types bct inner join transaction$charge$codes tcc on bct.id = tcc.CHARGE_CODE where tcc.TRANSACTION_CODE = 738', con=mysql_connection)
# df_charges = pd.read_sql('select bct.id,tcc.TRANSACTION_CODE,bct.CALCULATION_BASIS,bct.FLAT_AMOUNT,bct.CALC_TYPE,bct.PERCENTAGE,bct.UPTO_AMOUNT,bct.MINIMUM_AMOUNT,bct.MAXIMUM_AMOUNT,tcc.TAX from ft$commission$type bct inner join transaction$charge$codes tcc on bct.id = tcc.CHARGE_CODE where tcc.TRANSACTION_CODE = 738', con=mysql_connection)
# mysql_connection.close()
# df_charges.head()

In [ ]:
# cols_to_df_list = ['CALC_TYPE','PERCENTAGE','UPTO_AMOUNT','MINIMUM_AMOUNT','MAXIMUM_AMOUNT','id','TAX']
# df_chg = pd.DataFrame(columns = cols_to_df_list)
# for j in range (0,len(df_charges)):
#     df_len = len(df_chg)
#     for i in cols_to_df_list:
#         if df_charges[i][j]:
#             split_list = fn_str_split(df_charges[i][j])
#             for k in range (0,len(split_list)):
#                 if((i == "id") | (i == 'TAX')):
#                     df_chg.loc[df_len + k,i] = df_charges[i][j]
#                 else:
#                     df_chg.loc[df_len + k,i] = split_list[k]
# df_chg

In [ ]:
# df_chg['id'] = df_chg[['id']].ffill()
# df_chg['TAX'] = df_chg.groupby('id')['TAX'].ffill()
# df_chg

In [ ]:
# df_chg['PERCENTAGE'] = df_chg['PERCENTAGE'].astype('float')
# df_chg['MAX_AMT'] = df_chg['UPTO_AMOUNT'].astype('float')
# df_chg['MIN_CHARGE'] = df_chg['MINIMUM_AMOUNT'].astype('float')
# df_chg['MAX_CHARGE'] = df_chg['MAXIMUM_AMOUNT'].astype('float')
# df_chg['TAX'] = df_chg['TAX'].astype('float')
# df_chg

In [ ]:
# df_chg['MIN_AMT'] = 0
# for i in range(0,len(df_chg)):
#     if(i == 0):
#         df_chg['MIN_AMT'][i] =  0
#     else:
#         df_chg['MIN_AMT'][i] =  df_chg['MAX_AMT'][i-1] + 1

# df_chg.loc[(df_chg['MIN_AMT'].isnull()) | (df_chg['MIN_AMT'] > df_chg['MAX_AMT']),'MIN_AMT'] = 0
# df_chg = df_chg[["id","CALC_TYPE","PERCENTAGE","MIN_AMT","MAX_AMT","MIN_CHARGE","MAX_CHARGE","TAX"]]
# df_chg

In [ ]:
# df1 = pd.read_csv("sample.csv")
# df1.head()

In [ ]:
# # for expected charge initialization
# df1['EXPECTED_CHARGE'] = 0
# df1['EXPECTED_TAX'] = 0

# for i in range (0,len(df_chg)):
#     # for charge as per percentage
#     df1.loc[(df1['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df1['TRANSACTION_AMOUNT'] <= df_chg['MAX_AMT'][i]), 'CHARGE'] = round((df1['TRANSACTION_AMOUNT'] * df_chg['PERCENTAGE'][i] /100),2)
#     # for charge lesser then minimum charge amount
#     if(pd.isnull(df_chg.iloc[i]['MIN_CHARGE']) == False):
#         df1.loc[(df1['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df1['TRANSACTION_AMOUNT'] <= df_chg['MAX_AMT'][i]) & (df1['CHARGE'] < df_chg['MIN_CHARGE'][i]) , 'CHARGE'] =  df_chg['MIN_CHARGE'][i]
#     # for charge greter then minimum charge amount
#     if(pd.isnull(df_chg.iloc[i]['MAX_CHARGE']) == False):
#         df1.loc[(df1['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df1['TRANSACTION_AMOUNT'] <= df_chg['MAX_AMT'][i]) & (df1['CHARGE'] > df_chg['MAX_CHARGE'][i]) , 'CHARGE'] =  df_chg['MAX_CHARGE'][i]
#     # for tax as per percentage
#     if(pd.isnull(df_chg.iloc[i]['TAX']) == False):
#         df1.loc[(df1['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df1['TRANSACTION_AMOUNT'] <= df_chg['MAX_AMT'][i]), 'TAX'] = round((df1['CHARGE'] * df_chg['TAX'][i] /100),2)
    
#     # for expected charge
#     df1.loc[(df1['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df1['TRANSACTION_AMOUNT'] <= df_chg['MAX_AMT'][i]), 'EXPECTED_CHARGE'] = round((df1['EXPECTED_CHARGE'] + df1['CHARGE']),2)
#     # for expected tax
#     if(pd.isnull(df_chg.iloc[i]['TAX']) == False):
#         df1.loc[(df1['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df1['TRANSACTION_AMOUNT'] <= df_chg['MAX_AMT'][i]), 'EXPECTED_TAX'] = round((df1['EXPECTED_TAX'] + df1['TAX']),2)
    
# # # df_charges.loc[df_charges['MAX_AMT'].idxmax()]
# # # for mapping percentage of charge, greater then last upto amount
# # df1.loc[(df1['TRANSACTION_AMOUNT'] > df_charges.loc[df_charges['MAX_AMT'].idxmax()].MAX_AMT),'EXPECTED_CHARGE'] = round(df1['EXPECTED_CHARGE'] + (df1['TRANSACTION_AMOUNT'] * df_charges.loc[df_charges['MAX_AMT'].idxmax()].PERCENTAGE /100),2)
# # # for mapping min charge, greater then last upto amount
# # df1.loc[(df1['TRANSACTION_AMOUNT'] > df_charges.loc[df_charges['MAX_AMT'].idxmax()].MAX_AMT) & (df1['EXPECTED_CHARGE'] < df_charges.loc[df_charges['MAX_AMT'].idxmax()].MIN_CHARGE),'EXPECTED_CHARGE'] = df_charges.loc[df_charges['MAX_AMT'].idxmax()].MIN_CHARGE
# # # for mapping max charge, greater then last upto amount
# # df1.loc[(df1['TRANSACTION_AMOUNT'] > df_charges.loc[df_charges['MAX_AMT'].idxmax()].MAX_AMT) & (df1['EXPECTED_CHARGE'] > df_charges.loc[df_charges['MAX_AMT'].idxmax()].MAX_CHARGE),'EXPECTED_CHARGE'] = df_charges.loc[df_charges['MAX_AMT'].idxmax()].MAX_CHARGE
# df1.tail()

In [ ]:
# df_chg = df_chg.loc[(df_chg['MAX_AMT'].isnull())]
# df_chg.reset_index(drop=True, inplace=True)
# df_chg.head()

In [ ]:
# for i in range (0,len(df_chg)):
#     # for charge as per percentage
#     df1.loc[(df1['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'CHARGE'] = round((df1['TRANSACTION_AMOUNT'] * df_chg['PERCENTAGE'][i] /100),2)
#     # for charge lesser then minimum charge amount
#     if(pd.isnull(df_chg.iloc[i]['MIN_CHARGE']) == False):
#         df1.loc[(df1['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df1['CHARGE'] < df_chg['MIN_CHARGE'][i]) , 'CHARGE'] =  df_chg['MIN_CHARGE'][i]
#     # for charge greter then minimum charge amount
#     if(pd.isnull(df_chg.iloc[i]['MAX_CHARGE']) == False):
#         df1.loc[(df1['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df1['CHARGE'] > df_chg['MAX_CHARGE'][i]) , 'CHARGE'] =  df_chg['MAX_CHARGE'][i]
#     # for tax as per percentage
#     if(pd.isnull(df_chg.iloc[i]['TAX']) == False):
#         df1.loc[(df1['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'TAX'] = round((df1['CHARGE'] * df_chg['TAX'][i] /100),2)
    
#     # for expected charge
#     df1.loc[(df1['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'EXPECTED_CHARGE'] = round((df1['EXPECTED_CHARGE'] + df1['CHARGE']),2)
#     # for expected tax
#     if(pd.isnull(df_chg.iloc[i]['TAX']) == False):
#         df1.loc[(df1['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'EXPECTED_TAX'] = round((df1['EXPECTED_TAX'] + df1['TAX']),2)
# df1.tail()   

In [ ]:
# df1 = pd.read_csv("sample.csv")
# df1.head()

In [ ]:
# # for expected charge initialization
# df1['EXPECTED_CHARGE'] = 0
# df1['EXPECTED_TAX'] = 0

# for i in range (0,len(df_chg)):
#     # Chargable amount calculation
#     df1.loc[(df1['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'CHARGABLE_AMT'] = df1['TRANSACTION_AMOUNT'] - (df_chg['MIN_AMT'][i])
#     if(pd.isnull(df_chg.iloc[i]['MAX_AMT']) == False):
#         df1.loc[(df1['CHARGABLE_AMT'] > (df_chg['MAX_AMT'][i] - df_chg['MIN_AMT'][i])), 'CHARGABLE_AMT'] = df_chg['MAX_AMT'][i] - df_chg['MIN_AMT'][i]
#     else:
#         df1.loc[(df1['TRANSACTION_AMOUNT'] > (df_chg['MIN_AMT'][i])), 'CHARGABLE_AMT'] = df1['TRANSACTION_AMOUNT'] - (df_chg['MIN_AMT'][i])
#     # for charge as per percentage
#     df1.loc[(df1['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'CHARGE'] = round((df1['CHARGABLE_AMT'] * df_chg['PERCENTAGE'][i] /100),2)
#     # for tax as per percentage
#     if(pd.isnull(df_chg.iloc[i]['TAX']) == False):
#         df1.loc[(df1['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'TAX'] = round((df1['CHARGE'] * df_chg['TAX'][i] /100),2)

#     # for expected charge
#     df1.loc[(df1['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'EXPECTED_CHARGE'] = round((df1['EXPECTED_CHARGE'] + df1['CHARGE']),2)
#     # for expected tax
#     if(pd.isnull(df_chg.iloc[i]['TAX']) == False):
#         df1.loc[(df1['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'EXPECTED_TAX'] = round((df1['EXPECTED_TAX'] + df1['TAX']),2)
# df1.tail()